![](../images/rivacon_frontmark_combined_header.png)

In [ ]:
# Import necessary packages
from enum import Enum, unique

import pyvacon
import pyvacon.marketdata.testdata as mkt_testdata
import pyvacon.tools.enums as enums
import pyvacon.marketdata.bootstrapping as bootstr
import pyvacon.marketdata.plot as mkt_plot
import pyvacon.models.plot as model_plot
import pyvacon.models.tools as model_tools
import pyvacon.analytics as analytics
import pyvacon.tools.converter as converter

from matplotlib.lines import Line2D
from matplotlib.patches import Patch, Rectangle
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import matplotlib.transforms as mtransforms
%matplotlib inline

import datetime as dt
import math
import numpy as np
import ipywidgets as widgets

from scipy import stats
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from IPython.display import display_html
from IPython.display import HTML
# # Apply some CSS to this notebook
# HTML("""
# <style>
# //css
# </style>
# """)

# use ipynb to import function definitions from another notebook
import ipynb
from ipynb.fs.defs.ir_shock_scenarios import getShockedDiscountCurve

In [ ]:
# We define some constants which we'll use repeatedly throughout this notebook
notebook_is_draft = True
color_main = 'tab:blue'
color_highlight = 'tab:orange'
color_graphblue = 'rgb(78, 121, 167)'#'"#4e79a7"
color_graphblue_5 = 'rgba(78, 121, 167, 0.5)'
color_histmarker = "#4e79a7"
color_histmarkerborder = "White"
grid_alpha = 0.4
default_daycounter_type = enums.DayCounter.ACT365_FIXED
default_daycounter = analytics.DayCounter(default_daycounter_type)
default_interpolation_type = enums.InterpolationType.HAGAN_DF
default_extrapolation_type = enums.ExtrapolationType.NONE
default_plotly_scatter_mode = 'lines'
if notebook_is_draft:
    default_sample_size_MC = 500 # about the same sample size we have available for historical simulation
else:
    default_sample_size_MC = 10000
refdate = dt.datetime(year = 2019, month = 12, day = 30)

@unique
class UnitInterestRate(Enum):
    DECIMAL = 1
    PERCENT = 2
    BASISPOINTS = 3

In [ ]:
# Same for functions
def get_default_title_dict(title_text):
    return dict(
        text = title_text,
        yref = 'container',
        y = 0.9,
        xref = 'paper',
        x = 0.5,
        xanchor = 'center',
        yanchor = 'top')

def getDates(refdate, year_fractions):
    return [refdate + dt.timedelta(int(round(year_fractions[i]*365.25))) for i in range(len(year_fractions))]

def getDiscountFactors(
    interest_rates,
    year_fractions,
    unit_interest_rates
):
    if len(interest_rates) != len(year_fractions):
        raise Exception('You must supply an equal number of interest rates and year fractions!')
    
    if unit_interest_rates == UnitInterestRate.DECIMAL:
        factor_unit = 1
    elif unit_interest_rates == UnitInterestRate.PERCENT:
        factor_unit = 1/100
    elif unit_interest_rates == UnitInterestRate.BASISPOINTS:
        factor_unit = 1/(100*100)
    else:
        raise Exception('Value for parameter \'unit_interest_rates\' needs to be one of the values supplied by the Enum class \'UnitInterestRate\'!')
        
    dsc_fac = analytics.vectorDouble()
    for i in range(len(interest_rates)):
        dsc_fac.append(math.exp(-interest_rates[i]*factor_unit*year_fractions[i]))  
        
    return dsc_fac


def getDiscountCurve(
    object_name,
    refdate,
    dates,
    interest_rates,
    unit_interest_rates,
    daycounter_type = default_daycounter_type,
    interpolation_type = default_interpolation_type,
    extrapolation_type = default_extrapolation_type
):
    daycounter = analytics.DayCounter(daycounter_type)
    
    year_fractions = []
    for i in range(len(dates)):
        year_fractions.append(daycounter.yf(refdate, dates[i]))
    
    dsc_fac = getDiscountFactors(
        interest_rates,
        year_fractions,
        unit_interest_rates
    )
    
    discountCurve = analytics.DiscountCurve(
        object_name,
        refdate,
        dates,
        dsc_fac,
        daycounter_type,
        interpolation_type,
        extrapolation_type
    )
    
    return discountCurve


def getInterestRatesFromDiscountCurve(
    discount_curve,
    refdate,
    dates,
    unit_interest_rates
):
    if unit_interest_rates == UnitInterestRate.DECIMAL:
        factor_unit = 1
    elif unit_interest_rates == UnitInterestRate.PERCENT:
        factor_unit = 100
    elif unit_interest_rates == UnitInterestRate.BASISPOINTS:
        factor_unit = 100*100
    else:
        raise Exception('Value for parameter \'unit_interest_rates\' needs to be one of the values supplied by the Enum class \'UnitInterestRate\'!')
    
    daycounter = analytics.DayCounter(discount_curve.getDayCounterType())
    interest_rates = [ (-1) * math.log(discount_curve.value(dates[i], refdate)) / daycounter.yf(dates[i], refdate) * factor_unit for i in range(len(dates))]
    return interest_rates


def getBootstrappedData(
    raw_data,
    column_refdate,
    columns_maturity,
    maturities_yf,
    quotes_template,
    curve_name,
    daycounter_type,
    unit_interest_rates,
#     interpolation_type = default_interpolation_type,
#     extrapolation_type = default_extrapolation_type,
    discount_curves = None #,
#     basis_curves = None # TODO
):
    if unit_interest_rates == UnitInterestRate.DECIMAL:
        factor_unit = 1
    elif unit_interest_rates == UnitInterestRate.PERCENT:
        factor_unit = 100
    elif unit_interest_rates == UnitInterestRate.BASISPOINTS:
        factor_unit = 100*100
    else:
        raise Exception('Value for parameter \'unit_interest_rates\' needs to be one of the values supplied by the Enum class \'UnitInterestRate\'!')
    
    series_bootstrapped = []
        
    for i in range(0, len(raw_data.index)):
        row = raw_data[columns_maturity].iloc[i]

        # Copy the template and insert actual quotes
        quotes = quotes_template.copy(deep = True)
        quotes['Quote'] = row
        quotes['Quote'] = quotes['Quote'].apply(lambda x: x/factor_unit) # raw data is given in unit_interest_rates

        # set up curve parameters for bootstrapping algorithm
        refdate_curve = raw_data.iloc[i][column_refdate]
        
        if isinstance(refdate_curve, pd.Timestamp):
            refdate_curve_dt = refdate_curve.to_pydatetime()
        else:
            refdate_curve_dt = refdate_curve
            
        if not ( isinstance(refdate_curve_dt, dt.datetime) or isinstance(refdate_curve_dt, dt.date) ):
            raise Exception('The given reference dates need to be of type datetime.date, datetime.datetime or pandas.Timestamp')
            
        dates_curve = getDates(refdate_curve_dt, maturities_yf)
        
        if not isinstance(discount_curves, pd.DataFrame):
            discount_curve = None
        else:
#             if column_refdate not in discount_curves.columns:
#                 raise Exception('The DataFrame containing discount curves needs to supply the ref')
#             query = discount_curves[discount_curves['Analytics.DiscountCurve'].getRefDate() == refdate_curve]
            query = discount_curves[discount_curves[column_refdate] == refdate_curve]
            if query.empty:
                discount_curve = None
                # Throw error? Show warning?
            elif len(query.index) > 1:
                raise Exception('More than one discount curve was provided for reference date ' + refdate_curve_dt.strftime("%d-%m-%Y") + '. I don\'t know which one to use.')
            else:
                discount_curve = query['Analytics.DiscountCurve'].iloc[0]
        
        basis_curve = None # TODO
        
        curve_spec =  {
            'refDate': refdate_curve_dt, 
            'curveName': curve_name + refdate_curve_dt.strftime("%d-%m-%Y"),
            'dayCount': daycounter_type,
            'calendar': analytics.SimpleHolidayCalendar('GER_HOL'),
            'discountCurve': discount_curve,
            'basisCurve': basis_curve#,
#             'InterpolationType': interpolation_type,
#             'ExtrapolationType': extrapolation_type
        }

        # bootstrap the curve     
        curve_bootstrapped = bootstr.bootstrap_curve(quotes, curve_spec)

        # the resulting curve is given as a discount curve
        # -> compute the zero rates for the given maturities
        df = analytics.vectorDouble()
        dc = analytics.DayCounter(daycounter_type)
        curve_bootstrapped.value(df, refdate_curve_dt, dates_curve)
        data_bootstrapped = [-math.log(df[i])/dc.yf(refdate_curve_dt, dates_curve[i]) for i in range(len(df))]
        data_bootstrapped = [factor_unit*x for x in data_bootstrapped] # convert back to unit_interest_rates

        # put the data into a series and store it in a list
        series = pd.Series(data = [refdate_curve, curve_bootstrapped] + data_bootstrapped, index = [column_refdate, 'Analytics.DiscountCurve'] + columns_maturity)
        series_bootstrapped.append(series)
                
    return pd.DataFrame(data = series_bootstrapped)

# Introduction
## Value at risk
**Value at risk (VaR)** is a measure for the risk in a portfolio of financial assets. Given a time horizon of $n$ days and a confidence level $\alpha$, the VaR is the loss of value that has the probability $\alpha$ not to be exceeded within the next $n$ days. In other words, the VaR is the $\alpha$-quantile of the distribution of loss in the value of a portfolio over the next $n$ days.

The different methods for estimating the value at risk can be put into two major categories: Those using analytical models and those using simulations.

The goal of **analytical** methods is to define a probability distribution, which approximates the actual probability distribution of the portfolio value. One can then write down a closed formula for the value at risk.

**Simulation**-based methods simulate the change in value over the next $n$ days and use the resulting relative frequency distribution to 'read off' the value at risk.

In [ ]:
# Illustrate VaR using probability density of a normal distribution
def illustrateVaRNormalDistPDF(
    xmin,
    xmax,
    alpha,
    npoints = 100, # number of data points used to draw the density function
    mean = 0,
    std = 1
):
    normal_dist = stats.norm(mean,std)
    x_VaR = normal_dist.ppf(0.945)
    rangey = [0-0.03, normal_dist.pdf(mean) + 0.035]

    x_pdf = [xmin + i*(xmax - xmin)/(npoints - 1) for i in range(npoints)]
    y_pdf = [normal_dist.pdf(x) for x in x_pdf]#[math.exp(-x*x/2)/math.sqrt(2*math.pi) for x in x_density]

    x_alpha_annotation = x_VaR + 0.2*(xmax - x_VaR)
    x_fill_shape = [x_VaR + i*(xmax - x_VaR)/(npoints - 1) for i in range(npoints)]
    y_fill_shape = [normal_dist.pdf(x) for x in x_fill_shape]


    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x = x_fill_shape,
        y = y_fill_shape,
        mode = 'lines',
        showlegend = False,
        fill = 'tozeroy',
        marker = dict(color = color_graphblue)
    ))

    fig.add_trace(go.Scatter(
        x = x_pdf,
        y = y_pdf,
        name = 'probability density',
        mode = 'lines',
        showlegend = True,
        line=dict(color="Black"),
        opacity = 1
    ))

    fig.update_layout(
        showlegend=False,
        title = get_default_title_dict('Probability density')

        ,xaxis = dict(
            title_text = "Loss"
            ,showticklabels = False
        )
        ,yaxis = dict(
            title_text = "Frequency"
            ,showticklabels = False
            ,range = rangey
        )
        
        ,annotations = [
            dict(
                xref = 'x',
                x = x_VaR,
                yref = 'paper',
                y = -0.065,
                text = 'VaR',
                showarrow = False
            )

            ,dict(
                xref = 'x',
                x = x_alpha_annotation,
                yref = 'y',
                y = 0.5*normal_dist.pdf(x_alpha_annotation),
                text = '$1-\\alpha$',
                showarrow = False,
                font = dict(
                    size = 18
                )
            )
        ]

        ,shapes = [
            dict(
                type = 'line',
                xref = 'x',
                x0 = x_VaR,
                x1 = x_VaR,
                yref = 'y',
                y0 = rangey[0],
                y1 = normal_dist.pdf(x_VaR),
                line = dict(
                    color = color_histmarker
                )
            )
        ]
    )

    fig.show()

illustrateVaRNormalDistPDF(
    xmin = -3,
    xmax = 3,
    alpha = 0.945
)

In [ ]:
# Illustrate VaR using cumulative distribution function of a normal distribution
def illustrateVaRNormalDistCDF(
    xmin,
    xmax,
    alpha,
    npoints = 100, # number of data points used to draw the density function
    mean = 0,
    std = 1
):
    normal_dist = stats.norm(mean,std)
    x_VaR = normal_dist.ppf(0.945)

    x_cdf = [xmin + i*(xmax - xmin)/(npoints - 1) for i in range(npoints)]
    y_cdf = [normal_dist.cdf(x) for x in x_cdf]
    rangey = [-0.05, 1.05]


    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x = x_cdf,
        y = y_cdf,
        name = 'cumulative distribution',
        mode = 'lines',
        line=dict(color="Black"),
        opacity = 1
    ))

    fig.update_layout(
        showlegend=False,
        title = get_default_title_dict('Cumulative probability distribution')

        ,xaxis = dict(
            title_text = "Loss"
            ,showticklabels = False
        )

        ,yaxis = dict(
            title_text = "Cumulative probability",
            range = rangey
        )

        ,shapes = [

            # parallel to x, through alpha
            dict(
                type = 'line',
                xref = 'x',
                x0 = xmin,
                x1 = x_VaR,
                yref = 'y',
                y0 = alpha,
                y1 = alpha,
                line = dict(
                    color = color_graphblue
                )
            ),

            # parallel to y, through VaR
            dict(
                type = 'line',
                xref = 'x',
                x0 = x_VaR,
                x1 = x_VaR,
                yref = 'y',
                y0 = rangey[0],
                y1 = alpha,
                line = dict(
                    color = color_histmarker
                )
            )
        ]


        ,annotations = [
            dict(
                xref = 'x',
                x = x_VaR,
                yref = 'paper',
                y = -0.065,
                text = 'VaR',
                showarrow = False
            )

            ,dict(
                xref = 'paper',
                x = -0.02,
                yref = 'y',
                y = alpha,
                text = '$\\alpha$',
                showarrow = False,
                font = dict(
                    size = 15
                )
            )
        ]
    )

    fig.show()

illustrateVaRNormalDistCDF(
    xmin = -3,
    xmax = 3,
    alpha = 0.945
)

## Expected shortfall
By definition, VaR does not provide any information about the tail of the loss distribution. In fact, it is indifferent to the shape of the distribution beyond the chosen confidence level. For that reason, one often uses another risk measure in combination with VaR: The **expected shortfall (ES)**, also referred to as **conditional value at risk**, is the expected loss given that the loss exceeds VaR. 

## Historical simulation
A very popular way of simulating changes in value uses past market data to estimate what will happen in the future. To do so, we first have to identify all market variables affecting the portfolio value. Then we collect data on how these variables moved over the past $k+n$ days. This allows us to calculate $k$ historical scenarios of what can happen in $n$ days. Assuming that the market will behave in the future as it did in the past, we can compute the portfolio value in each of these scenarios. This provides us with a relative frequency distribution, which we then use to determine value at risk and expected shortfall.

## Monte Carlo simulation
Monte Carlo simulation is similar to historical simulation in the sense that we also
- generate a set of market scenarios,
- compute the value of our portfolio in each of these scenarios and
- use the resulting relative frequency distribution to determine the value at risk.

They differ in the method for generating market scenarios: Instead of historical data, Monte Carlo simulation uses randomly generated movements of all relevant market variables. This requires more work (for example, you first have to develop a model for the market movements), but also comes with more flexibility.

## This notebook
We are going to look at two rather simple portfolios, one containing only a single bond and one containing a swap in addition to that bond. Their values can be computed by summing over all discounted future cash flows of said bond (and swap). Therefore, the only market variables affecting the portfolio values are the interest rates we use to determine the discount factors.
We will use both historical and Monte Carlo simulation to obtain interest rate scenarios. Based on these scenarios, we are going to determine value at risk and expected shortfall for both portfolios.

# Historical simulation
## Historical data
We choose to discount future cash flows using EONIA interest rate curves. We have historical data from every business day of 2018 and 2019 available to us. The data includes rates for various maturities. We'll load the data for maturities of 1 day, 1-11 months and 1-10 years.

*Note: These interest rates are not zero-coupon rates. We will determine the zero rates in [the bootstrapping section below](#bootstrapping).*

In [ ]:
# load test data from an Excel file
xl = pd.ExcelFile('TestDaten.xlsx')
#print(xl.sheet_names)

In [ ]:
# import data into pandas dataframe
data_EONIA_raw = xl.parse('EONIA')
data_EUR3M_raw = xl.parse('EUR3M')
data_EUR6M_raw = xl.parse('EUR6M')
del xl
columns_maturity_EONIA = [
   '1D',
   '1M',
   '2M',
   '3M',
   '4M',
   '5M',
   '6M',
   '7M',
   '8M',
   '9M',
   '10M',
   '11M',
   '1Y',
   '2Y',
   '3Y',
   '4Y',
   '5Y',
   '6Y',
   '7Y',
   '8Y',
   '9Y',
   '10Y'
]
columns_maturity_EUR3M = [
   '3M',
   '6M',
   '9M',
   '1Y',
   '2Y',
   '3Y',
   '4Y',
   '5Y',
   '6Y',
   '7Y',
   '8Y',
   '9Y',
   '10Y'
]
columns_maturity_EUR6M = [
   '6M',
   '1Y',
   '2Y',
   '3Y',
   '4Y',
   '5Y',
   '6Y',
   '7Y',
   '8Y',
   '9Y',
   '10Y'
]

data_EONIA_raw = pd.DataFrame(data_EONIA_raw, columns = ['RefDate'] + columns_maturity_EONIA)
data_EUR3M_raw = pd.DataFrame(data_EUR3M_raw, columns = ['RefDate'] + columns_maturity_EUR3M)
data_EUR6M_raw = pd.DataFrame(data_EUR6M_raw, columns = ['RefDate'] + columns_maturity_EUR6M)
# print(['RefDate'] + columns_maturity_EONIA)

# convert Excel dates to a more useful format
data_EONIA_raw['RefDate'] = pd.TimedeltaIndex(data_EONIA_raw['RefDate'], unit='d') + dt.datetime(1899, 12, 30)
data_EUR3M_raw['RefDate'] = pd.TimedeltaIndex(data_EUR3M_raw['RefDate'], unit='d') + dt.datetime(1899, 12, 30)
data_EUR6M_raw['RefDate'] = pd.TimedeltaIndex(data_EUR6M_raw['RefDate'], unit='d') + dt.datetime(1899, 12, 30)
#display(data_EONIA.head(5))
#display(data_EONIA.tail(5))

Since we'll need them later, we store the selected maturities in the form of year fractions.

In [ ]:
# maturities in years
maturities_EONIA_yf = [1/365] # 1 day
maturities_EONIA_yf.extend( (np.arange(11)+1)/12 ) # 1 to 11 months
maturities_EONIA_yf.extend(np.arange(10)+1) # 1 to 10 years

maturities_EUR3M_yf = [3/12, 6/12, 9/12] # 3, 6 and 9 months
maturities_EUR3M_yf.extend(np.arange(10)+1) # 1 to 10 years

maturities_EUR6M_yf = [6/12] # 6 months
maturities_EUR6M_yf.extend(np.arange(10)+1) # 1 to 10 years


maturities_EONIA_dates = getDates(refdate, maturities_EONIA_yf)
maturities_EUR3M_dates = getDates(refdate, maturities_EUR3M_yf)
maturities_EUR6M_dates = getDates(refdate, maturities_EUR6M_yf)

<a id='bootstrapping'></a>
### Bootstrapping
#### EONIA

In [ ]:
# The pyvacon bootstrap algorithm needs the quotes of the curve we want to boostrap to be provided
# via a DataFrame that has a particular structure. We will now build such a DataFrame with all quotes
# set to NaN. We will insert actual quotes before bootstrapping.
ncols = len(columns_maturity_EONIA)
dfQuotes_EONIA_template = pd.DataFrame(data = columns_maturity_EONIA, index = columns_maturity_EONIA, columns = ['Maturity'])
tenors = ['1D', '1M', '2M', '3M', '4M', '5M', '6M', '7M', '8M', '9M', '10M', '11M'] + 10 * ['1Y']
dfQuotes_EONIA_template['Instrument'] = ['DEPOSIT'] + ( (ncols-1)*['OIS'] )
dfQuotes_EONIA_template['Quote'] = ncols*['NaN']# will be replaced with actual data later
dfQuotes_EONIA_template['Currency'] = ncols*['EUR']
dfQuotes_EONIA_template['UnderlyingIndex'] = ncols*['EONIA']
dfQuotes_EONIA_template['UnderlyingTenor'] = tenors
dfQuotes_EONIA_template['UnderlyingPaymentFrequency'] = tenors
dfQuotes_EONIA_template['BasisIndex'] = ncols*['NaN']
dfQuotes_EONIA_template['BasisTenor'] = ncols*['NaN']
dfQuotes_EONIA_template['BasisPaymentFrequency'] = ncols*['NaN']
dfQuotes_EONIA_template['PaymentFrequencyFixed'] = tenors
dfQuotes_EONIA_template['DayCountFixed'] = ncols*['Act360']
dfQuotes_EONIA_template['DayCountFloat'] = ncols*['Act360']
dfQuotes_EONIA_template['DayCountBasis'] = ncols*['NaN']
dfQuotes_EONIA_template['RollConventionFixed'] = ncols*['ModifiedFollowing']
dfQuotes_EONIA_template['RollConventionFloat'] = ncols*['ModifiedFollowing']
dfQuotes_EONIA_template['RollConventionBasis'] = ncols*['NaN']
dfQuotes_EONIA_template['SpotLag'] = ['0D'] + ((ncols-1)*['2D'])
del ncols
# dfQuotes_EONIA_template.head(999)

In [ ]:
# bootstrap the data and put it into a DataFrame
data_EONIA_bootstrapped = getBootstrappedData(
    data_EONIA_raw,
    'RefDate',
    columns_maturity_EONIA,
    maturities_EONIA_yf,
    dfQuotes_EONIA_template,
    'EONIA',
    enums.DayCounter.ACT360,
    UnitInterestRate.PERCENT
)

In [ ]:
# compare raw and bootstrapped curves
dc = analytics.DayCounter(enums.DayCounter.ACT360)
if(notebook_is_draft):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x = maturities_EONIA_yf, y = data_EONIA_raw[columns_maturity_EONIA].iloc[1], name = 'raw', mode = default_plotly_scatter_mode))
    fig.add_trace(go.Scatter(x = maturities_EONIA_yf, y = data_EONIA_bootstrapped[columns_maturity_EONIA].iloc[1], name = 'bootstrapped', mode = default_plotly_scatter_mode))

#     fig.add_trace(go.Scatter(x = maturities_EONIA_yf, y = data_EONIA_raw[columns_maturity_EONIA].iloc[12], name = 'raw', mode = default_plotly_scatter_mode))
#     fig.add_trace(go.Scatter(x = maturities_EONIA_yf, y = data_EONIA_bootstrapped[columns_maturity_EONIA].iloc[12], name = 'bootstrapped', mode = default_plotly_scatter_mode))


    fig.update_layout(
        showlegend=True,
        xaxis = dict(title_text = "Expiry (in years)"),
        yaxis = dict(title_text = "Interest rate (in percent)")
    )

    fig.show()

#del series_bootstrapped
del dc

In [ ]:
# use the bootstrapped data from here on out
data_EONIA = data_EONIA_bootstrapped

#### EURIBOR 3M

In [ ]:
# do the same for EUR3M
ncols = len(columns_maturity_EUR3M)
dfQuotes_EUR3M_template = pd.DataFrame(data = columns_maturity_EUR3M, index = columns_maturity_EUR3M, columns = ['Maturity'])
dfQuotes_EUR3M_template['Instrument'] = ['DEPOSIT'] + ( (ncols-1)*['IRS'] )
dfQuotes_EUR3M_template['Quote'] = ncols*['NaN']# will be replaced with actual data later
dfQuotes_EUR3M_template['Currency'] = ncols*['EUR']
dfQuotes_EUR3M_template['UnderlyingIndex'] = ncols*['EURIBOR']
dfQuotes_EUR3M_template['UnderlyingTenor'] = ncols*['3M']
dfQuotes_EUR3M_template['UnderlyingPaymentFrequency'] = ncols*['3M']
dfQuotes_EUR3M_template['BasisIndex'] = ncols*['NaN']
dfQuotes_EUR3M_template['BasisTenor'] = ncols*['NaN']
dfQuotes_EUR3M_template['BasisPaymentFrequency'] = ncols*['NaN']
dfQuotes_EUR3M_template['PaymentFrequencyFixed'] = ['3M', '6M', '9M'] + ( (ncols-3)*['1Y'] )
dfQuotes_EUR3M_template['DayCountFixed'] = ncols*['Act360']
dfQuotes_EUR3M_template['DayCountFloat'] = ncols*['Act360']
dfQuotes_EUR3M_template['DayCountBasis'] = ncols*['NaN']
dfQuotes_EUR3M_template['RollConventionFixed'] = ncols*['ModifiedFollowing']
dfQuotes_EUR3M_template['RollConventionFloat'] = ncols*['ModifiedFollowing']
dfQuotes_EUR3M_template['RollConventionBasis'] = ncols*['NaN']
dfQuotes_EUR3M_template['SpotLag'] = ncols*['2D']
del ncols
# dfQuotes_EUR3M_template.head(999)

# bootstrap the data and put it into a DataFrame
data_EUR3M_bootstrapped = getBootstrappedData(
    data_EUR3M_raw,
    'RefDate',
    columns_maturity_EUR3M,
    maturities_EUR3M_yf,
    dfQuotes_EUR3M_template,
    'EUR3M',
    enums.DayCounter.ACT360,
    UnitInterestRate.PERCENT,
    data_EONIA_bootstrapped[['RefDate', 'Analytics.DiscountCurve']]
)

In [ ]:
# compare raw and bootstrapped curves
dc = analytics.DayCounter(enums.DayCounter.ACT360)
if(notebook_is_draft):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x = maturities_EUR3M_yf, y = data_EUR3M_raw[columns_maturity_EUR3M].iloc[1], name = 'raw', mode = default_plotly_scatter_mode))
    fig.add_trace(go.Scatter(x = maturities_EUR3M_yf, y = data_EUR3M_bootstrapped[columns_maturity_EUR3M].iloc[1], name = 'bootstrapped', mode = default_plotly_scatter_mode))

#     fig.add_trace(go.Scatter(x = maturities_EUR3M_yf, y = data_EUR3M_raw[columns_maturity_EUR3M].iloc[12], name = 'raw', mode = default_plotly_scatter_mode))
#     fig.add_trace(go.Scatter(x = maturities_EUR3M_yf, y = data_EUR3M_bootstrapped[columns_maturity_EUR3M].iloc[12], name = 'bootstrapped', mode = default_plotly_scatter_mode))

    fig.update_layout(
        showlegend=True,
        xaxis = dict(title_text = "Expiry (in years)"),
        yaxis = dict(title_text = "Interest rate (in percent)")
    )

    fig.show()

#del series_bootstrapped
del dc

In [ ]:
# use the bootstrapped data from here on out
data_EUR3M = data_EUR3M_bootstrapped

#### EURIBOR 6M

In [ ]:
# do the same for EUR6M
ncols = len(columns_maturity_EUR6M)
dfQuotes_EUR6M_template = pd.DataFrame(data = columns_maturity_EUR6M, index = columns_maturity_EUR6M, columns = ['Maturity'])
dfQuotes_EUR6M_template['Instrument'] = 0*['DEPOSIT'] + ( (ncols-0)*['IRS'] )#ncols*['IRS']#
dfQuotes_EUR6M_template['Quote'] = ncols*['NaN']# will be replaced with actual data later
dfQuotes_EUR6M_template['Currency'] = ncols*['EUR']
dfQuotes_EUR6M_template['UnderlyingIndex'] = ncols*['EURIBOR']
dfQuotes_EUR6M_template['UnderlyingTenor'] = ncols*['6M']
dfQuotes_EUR6M_template['UnderlyingPaymentFrequency'] = ncols*['6M']
dfQuotes_EUR6M_template['BasisIndex'] = ncols*['NaN']
dfQuotes_EUR6M_template['BasisTenor'] = ncols*['NaN']
dfQuotes_EUR6M_template['BasisPaymentFrequency'] = ncols*['NaN']
dfQuotes_EUR6M_template['PaymentFrequencyFixed'] = ['6M'] + ( (ncols-1)*['1Y'] )
dfQuotes_EUR6M_template['DayCountFixed'] = ncols*['Act360']
dfQuotes_EUR6M_template['DayCountFloat'] = ncols*['Act360']
dfQuotes_EUR6M_template['DayCountBasis'] = ncols*['NaN']
dfQuotes_EUR6M_template['RollConventionFixed'] = ncols*['ModifiedFollowing']
dfQuotes_EUR6M_template['RollConventionFloat'] = ncols*['ModifiedFollowing']
dfQuotes_EUR6M_template['RollConventionBasis'] = ncols*['NaN']
dfQuotes_EUR6M_template['SpotLag'] = ncols*['2D']
del ncols
# dfQuotes_EUR6M_template.head(999)

# bootstrap the data and put it into a DataFrame
data_EUR6M_bootstrapped = getBootstrappedData(
    data_EUR6M_raw,
    'RefDate',
    columns_maturity_EUR6M,
    maturities_EUR6M_yf,
    dfQuotes_EUR6M_template,
    'EUR6M',
    enums.DayCounter.ACT360,
    UnitInterestRate.PERCENT,
    data_EONIA_bootstrapped[['RefDate', 'Analytics.DiscountCurve']]
)

In [ ]:
# compare raw and bootstrapped curves
dc = analytics.DayCounter(enums.DayCounter.ACT360)
if(notebook_is_draft):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x = maturities_EUR6M_yf, y = data_EUR6M_raw[columns_maturity_EUR6M].iloc[1], name = 'raw', mode = default_plotly_scatter_mode))
    fig.add_trace(go.Scatter(x = maturities_EUR6M_yf, y = data_EUR6M_bootstrapped[columns_maturity_EUR6M].iloc[1], name = 'bootstrapped', mode = default_plotly_scatter_mode))

#     fig.add_trace(go.Scatter(x = maturities_EUR6M_yf, y = data_EUR6M_raw[columns_maturity_EUR6M].iloc[12], name = 'raw', mode = default_plotly_scatter_mode))
#     fig.add_trace(go.Scatter(x = maturities_EUR6M_yf, y = data_EUR6M_bootstrapped[columns_maturity_EUR6M].iloc[12], name = 'bootstrapped', mode = default_plotly_scatter_mode))

    fig.update_layout(
        showlegend=True,
        xaxis = dict(title_text = "Expiry (in years)"),
        yaxis = dict(title_text = "Interest rate (in percent)")
    )

    fig.show()

#del series_bootstrapped
del dc

In [ ]:
# use the bootstrapped data from here on out
data_EUR6M = data_EUR6M_bootstrapped

### Differences in basis

In [ ]:
# define a function that computes the differences in interest rates for common reference dates and maturities
def computeBasisSpreads(
    data1,
    data2,
    columns_maturity1,
    columns_maturity2,
    column_refdate1,
    column_refdate2
):
    common_dates = [d for d in data1[column_refdate1].tolist() if d in data2[column_refdate2].tolist()]

    series = []

    for adate in common_dates:
        row1 = data1[data1[column_refdate1] == adate][columns_maturity1]
        row2 = data2[data2[column_refdate2] == adate][columns_maturity2]
        
        if len(row1.index) != 1:
            raise Exception('Found an unexpected number of rows in data set number 1 for reference date ' + adate.strftime("%d-%m-%Y") + '. Expected 1, found ' + str(len(row1.index)) + '.')

        if len(row2.index) != 1:
            raise Exception('Found an unexpected number of rows in data set number 2 for reference date ' + adate.strftime("%d-%m-%Y") + '. Expected 1, found ' + str(len(row2.index)) + '.')

        common_maturities = [c for c in row1.columns if c in row2.columns]
        diff = row2[common_maturities].iloc[0] - row1[common_maturities].iloc[0]

        series.append(pd.Series(data = [adate]+diff.tolist(), index = [column_refdate1] + diff.index.tolist()))

    return pd.DataFrame(data = series)

#### EUR3M-EONIA

In [ ]:
spreads_EUR3M_EONIA = computeBasisSpreads(
    data1 = data_EUR3M_bootstrapped,
    data2 = data_EONIA_bootstrapped,
    columns_maturity1 = columns_maturity_EUR3M,
    columns_maturity2 = columns_maturity_EONIA,
    column_refdate1 = 'RefDate',
    column_refdate2 = 'RefDate'
)
columns_maturity_EUR3M_EONIA = [c for c in columns_maturity_EUR3M if c in columns_maturity_EONIA]
maturities_EUR3M_EONIA_yf = [yf for yf in maturities_EUR3M_yf if yf in maturities_EONIA_yf] # TODO: find better way of doing this
maturities_EUR3M_EONIA_dates = getDates(refdate, maturities_EUR3M_EONIA_yf)

#### EUR3M-EUR6M

In [ ]:
spreads_EUR3M_EUR6M = computeBasisSpreads(
    data1 = data_EUR3M_bootstrapped,
    data2 = data_EUR6M_bootstrapped,
    columns_maturity1 = columns_maturity_EUR3M,
    columns_maturity2 = columns_maturity_EUR6M,
    column_refdate1 = 'RefDate',
    column_refdate2 = 'RefDate'
)
columns_maturity_EUR3M_EUR6M = [c for c in columns_maturity_EUR3M if c in columns_maturity_EUR6M]
maturities_EUR3M_EUR6M_yf = [yf for yf in maturities_EUR3M_yf if yf in maturities_EUR6M_yf] # TODO: find better way of doing this
maturities_EUR3M_EUR6M_dates = getDates(refdate, maturities_EUR3M_EUR6M_yf)

## Scenario generation
As mentioned in the introduction, our goal is to use historical data to simulate how much the relevant market variables might change from now to $n$ days from now.

### Example 1
Let's assume that $n=1$. In that case, we are asking how much a given market variable can change from one business day to the next.
We assume that our historical data is ordered by date ascending and numbered consecutively, starting at 1. If $v_i$ denotes the value of the market variable on day $i$, then we can compute change scenarios in the following way.

| Scenario | From | To | Absolute change | Relative change |
| :---: | :---: | :---: | :---: | :---: |
| 1 | Day 1 | Day 2 | $d_1 = v_2 - v_1$ | $q_1 = \frac{v_2}{v_1}$ |
| 2 | Day 2 | Day 3 | $d_2 = v_3 - v_2$ | $q_2 = \frac{v_3}{v_2}$ |
| 3 | Day 3 | Day 4 | $d_3 = v_4 - v_3$ | $q_3 = \frac{v_4}{v_3}$ |
| 4 | Day 4 | Day 5 | $d_4 = v_5 - v_4$ | $q_4 = \frac{v_5}{v_4}$ |
| ... | ||||

After we compute these change scenarios (or shift scenarios), we can apply them to the current value $v$ of the market variable to obtain market scenarios: We can either add the absolute changes to the current value...

| Scenario | Value of market variable |
| :---: | :---: | 
| 1 | $v + d_1$ | 
| 2 | $v + d_2$ | 
| 3 | $v + d_3$ | 
| 4 | $v + d_4$ |
| ... | |

... or multiply the current value by the relative changes

| Scenario | Value of market variable |
| :---: | :---: | 
| 1 | $v \cdot q_1$ |
| 2 | $v \cdot q_2$ | 
| 3 | $v \cdot q_3$ | 
| 4 | $v \cdot q_4$ |
| ... | |

Which of these approaches you choose should depend on the considered market variable. In the case of interest rates, it turns out that using absolute changes produces more realistic scenarios than using relative changes.

### Example 2
Note that, since we have one data point for every business day, the way we computed the change scenarios in Example 1 seemed very natural. If we now let $n=10$, we have to think about it more carefully. Consider the following two approaches.

*Approach 1*

We compute the change in value from day $i$ to day $i+10$ for **all days** where that is possible.

| Scenario | From | To | Absolute Change | Relative Change |
| :---: | :---: | :---: | :---: | :---: |
| 1 | Day 1 | Day 11 | $v_{11} - v_1$ | $\frac{v_{11}}{v_1}$ |
| 2 | Day 2 | Day 12 | $v_{12} - v_2$ | $\frac{v_{12}}{v_2}$ |
| 3 | Day 3 | Day 13 | $v_{13} - v_3$ | $\frac{v_{13}}{v_3}$ |
| 4 | Day 4 | Day 14 | $v_{14} - v_4$ | $\frac{v_{14}}{v_4}$ |
| ... | ||||

You'll find that this leads to significant **overlap in the time frames** (From -> To) behind the scenarios. The time frames of scenario 2 and scenario 4, for example, overlap in days 4 to 12. This introduces **correlation** between the scenarios.

Remark: This is an example of **autocorrelation**. In the context of time series, autocorrelation is a measure of the similarity between values of one and the same variable at different points in time. In our example that variable is the change in interest rates in the last 10 days. By choosing overlapping time frames, each data point has an influence on multiple values in the time series. This leads to correlation between the values. The bigger the overlap, the stronger the correlation.


*Approach 2*

To avoid this effect, we can choose the time frames such that they have less or no overlap.

| Scenario | From | To | Absolute Change | Relative Change |
| :---: | :---: | :---: | :---: | :---: |
| 1 | Day 1 | Day 11 | $v_{11} - v_{1}$ | $\frac{v_{11}}{v_1}$ |
| 2 | Day 11 | Day 21 | $v_{21} - v_{11}$ | $\frac{v_{21}}{v_{11}}$ |
| 3 | Day 21 | Day 31 | $v_{31} - v_{21}$ | $\frac{v_{31}}{v_{21}}$ |
| 4 | Day 31 | Day 41 | $v_{41} - v_{31}$ | $\frac{v_{41}}{v_{31}}$ |
| ... | ||||

As a consequence, we end up with only about **a tenth the number of scenarios** we had in Approach 1. Of course, we can try to get more data, but that can be expensive or simply not possible (especially, if you consider time frames spanning a whole year, as is often the case in practice). Furthermore, one can argue that data becomes less relevant the further it reaches into the past. 

### What we do in this notebook
The following code is generic in the sense that you can freely choose the time horizon $n$ and whether you want the scenarios to be computed using absolute or relative changes. However, the amount of overlap in the time frames can currently not be controlled.

Assuming that our historical data is ordered by date ascending and numbered consecutively, let $n$ be the selected time horizon in days, $v$ be the current value of a market variable and $v_i$ the value it had on date $i$. Then we'll compute the value $s_i$ of the market variable in the $i$-th scenario as either
$$s_i = v + (v_{i+n} - v_i)$$
or
$$s_i = v \cdot \frac{v_{i+n}}{v_i}$$

In [ ]:
timehorizon = 1 # number of business days
scenario_construction_type = 'absolute' # absolute or relative

*Note: You can change these parameters to your liking and rerun the code to see the effects. You can use this to verify that the interest rate scenarios generated by applying relative changes can be a bit unrealistic.*

In [ ]:
# define a function that computes the absolute or relative differences
def computeScenarios(
    data,
    column_refdate,
    columns_maturity,
    ndays,
    absolute = True
):
    # Sort by refdate descending
    data_sorted = data.sort_values(column_refdate, ascending = False)
    
    # Restrict to the columns containing interest rates
    data_rates_only = pd.DataFrame(data = data_sorted, columns = columns_maturity)
    
    # Copy the data frame structure
    data_scenarios = pd.DataFrame().reindex_like(data_rates_only)
    
    # Compute the absolute or relative changes over n days
    for i in range(len(data_scenarios.index)-ndays):
        if absolute:
            data_scenarios.iloc[i+ndays, :] = data_rates_only.iloc[i, :] - data_rates_only.iloc[i+ndays, :]
        else:
            if data_rates_only.iloc[i+ndays, :] != 0:
                data_scenarios.iloc[i+ndays, :] = data_rates_only.iloc[i, :] / data_rates_only.iloc[i+ndays, :]
    
    # Remove the rows containing NaN (i.e. the first n rows and those where we divided by 0)
    return data_scenarios.dropna()

In [ ]:
# define a function that applies the scenarios to given interest rates
def applyScenarios(
    current,
    scenarios,
    absolute = True
):
    if not isinstance(current, pd.Series):
        raise Exception('The current data needs to be provided in a pandas.Series.')
    if not isinstance(scenarios, pd.DataFrame):
        raise Exception('Scenarios need to be provided in a pandas.DataFrame.')
    if not set(current.index) == set(scenarios.columns):
        raise Exception('The names of the columns of the current data and the scenario data need to match.')
    
    applied = pd.DataFrame().reindex_like(scenarios)
    for i in applied.index:
        if absolute:
            applied.loc[i] = current + scenarios.loc[i]
        else:
            applied.loc[i] = current * scenarios.loc[i]
    return applied


# applyScenarios(
#     pd.Series(data = [1,2,3,4,5]),
#     pd.DataFrame(data = [
#         [1,1,1,1,1],
#         [2,2,2,2,2],
#         [1,2,3,4,5],
#         [0.1, 0.1, 0.1, 0.1, 0.1]
#     ]),
#     False
# ).head()

### EONIA

In [ ]:
# EONIA
# save the current market data in a pandas.series
data_EONIA_current = data_EONIA.iloc[0,:][columns_maturity_EONIA]

# compute and apply scenarios
data_scenarios_EONIA_diff = computeScenarios(
    data_EONIA,
    'RefDate',
    columns_maturity_EONIA,
    timehorizon,
    absolute = (scenario_construction_type == 'absolute')
)

data_scenarios_EONIA = applyScenarios(
    data_EONIA_current,
    data_scenarios_EONIA_diff,
    (scenario_construction_type == 'absolute')
)

### EUR3M

In [ ]:
# EUR3M
# save the current market data in a pandas.series
data_EUR3M_current = data_EUR3M.iloc[0,:][columns_maturity_EUR3M]

# compute and apply scenarios
data_scenarios_EUR3M_diff = computeScenarios(
    data_EUR3M,
    'RefDate',
    columns_maturity_EUR3M,
    timehorizon,
    absolute = (scenario_construction_type == 'absolute')
)

data_scenarios_EUR3M = applyScenarios(
    data_EUR3M_current,
    data_scenarios_EUR3M_diff,
    (scenario_construction_type == 'absolute')
)

### EUR6M

In [ ]:
# EUR6M
# save the current market data in a pandas.series
data_EUR6M_current = data_EUR6M.iloc[0,:][columns_maturity_EUR6M]

# compute and apply scenarios
data_scenarios_EUR6M_diff = computeScenarios(
    data_EUR6M,
    'RefDate',
    columns_maturity_EUR6M,
    timehorizon,
    absolute = (scenario_construction_type == 'absolute')
)

data_scenarios_EUR6M = applyScenarios(
    data_EUR6M_current,
    data_scenarios_EUR6M_diff,
    (scenario_construction_type == 'absolute')
)

### EUR3M-EONIA-spreads

In [ ]:
# EUR3M-EONIA-spreads
# save the current market data in a pandas.series
spreads_EUR3M_EONIA_current = spreads_EUR3M_EONIA.iloc[0,:][columns_maturity_EUR3M_EONIA]

# compute and apply scenarios
spreads_scenarios_EUR3M_EONIA_diff = computeScenarios(
    spreads_EUR3M_EONIA,
    'RefDate',
    columns_maturity_EUR3M_EONIA,
    timehorizon,
    absolute = (scenario_construction_type == 'absolute')
)

spreads_scenarios_EUR3M_EONIA = applyScenarios(
    spreads_EUR3M_EONIA_current,
    spreads_scenarios_EUR3M_EONIA_diff,
    (scenario_construction_type == 'absolute')
)

In [ ]:
# Check consistency of EUR3M-EONIA spreads
diff = data_scenarios_EONIA[columns_maturity_EUR3M_EONIA] - (data_scenarios_EUR3M[columns_maturity_EUR3M_EONIA] + spreads_scenarios_EUR3M_EONIA[columns_maturity_EUR3M_EONIA])
display(diff.describe())
del diff

### EUR3M-EUR6M-spreads

In [ ]:
# EUR3M-EUR6M-spreads
# save the current market data in a pandas.series
spreads_EUR3M_EUR6M_current = spreads_EUR3M_EUR6M.iloc[0,:][columns_maturity_EUR3M_EUR6M]

# compute and apply scenarios
spreads_scenarios_EUR3M_EUR6M_diff = computeScenarios(
    spreads_EUR3M_EUR6M,
    'RefDate',
    columns_maturity_EUR3M_EUR6M,
    timehorizon,
    absolute = (scenario_construction_type == 'absolute')
)

spreads_scenarios_EUR3M_EUR6M = applyScenarios(
    spreads_EUR3M_EUR6M_current,
    spreads_scenarios_EUR3M_EUR6M_diff,
    (scenario_construction_type == 'absolute')
)

In [ ]:
# Check consistency of EUR3M-EUR6M-spreads
diff = data_scenarios_EUR6M[columns_maturity_EUR3M_EUR6M] - (data_scenarios_EUR3M[columns_maturity_EUR3M_EUR6M] + spreads_scenarios_EUR3M_EUR6M[columns_maturity_EUR3M_EUR6M])
display(diff.describe())
del diff

## Plot Scenarios
To get a sense of how different the generated scenarios are from the current data, we plot all of them and highlight the ones that are (in a certain sense) the 'most distant'.

In [ ]:
# define a function that plots the given scenarios and highlights given rows
def plotScenarios(
    data_current,
    data_scenarios,
    maturities_yf,
    title_figure,
    label_curves_bulk,
    label_curves_highlighted = 'extreme scenarios',
    highlighted_indeces = [],
    integer_index = True,
    use_matplotlib = False
):
    if use_matplotlib:
        fig = plt.figure(figsize=(16,8))
        ax = fig.gca()

        color_current = 'w'
        color_bulk = 'k'
        color_maxdist = 'tab:blue'
        ax.plot(maturities_yf, data_current, '.-', label = 'current curve', color = color_current, zorder = 20)
        
        alpha = 20/len(data_scenarios.index)#0.05
        if integer_index:
            ax.plot(maturities_yf, data_scenarios.iloc[[x for x in range(len(data_scenarios.index)) if x not in highlighted_indeces], :].transpose(), '.-', label = 'other scenarios', color = color_bulk, zorder = 15, alpha=alpha)
            if highlighted_indeces != []:
                ax.plot(maturities_yf, data_scenarios.iloc[highlighted_indeces, :].transpose(), '.-', label = 'extreme scenarios', color = color_maxdist, zorder = 15, alpha=1)
        else:
            ax.plot(maturities_yf, data_scenarios.loc[~data_scenarios.index.isin(highlighted_indeces), :].transpose(), '.-', label = 'other scenarios', color = color_bulk, zorder = 15, alpha=0.05)
            ax.plot(maturities_yf, data_scenarios.loc[data_scenarios.index.isin(highlighted_indeces), :].transpose(), '.-', label = 'extreme scenarios', color = color_maxdist, zorder = 15, alpha=1)
        
        
        plt.xlabel('Expiry (in years)')
        plt.ylabel('Interest rate (in percent)')
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))

        legend_elements = [
            Patch(facecolor=color_current, edgecolor='gainsboro', label='current curve'),
            Patch(facecolor=color_bulk, label=label_curves_bulk)
        ]
        if highlighted_indeces != []:
            legend_elements += [Patch(facecolor=color_maxdist, label='extreme scenarios')]
        plt.legend(handles=legend_elements, loc='lower right')

        plt.show()

        
    # use plotly
    else:
        fig = go.Figure()
        
        if integer_index:
            forin = [i for i in range(len(data_scenarios.index)) if i not in highlighted_indeces]
        else:
            forin = [i for i in data_scenarios.index if i not in highlighted_indeces]

        opacity = max(0.001, min(1, 25/len(data_scenarios.index)))
        showlegend = True
        for i in forin:
            if integer_index:
                row = data_scenarios.iloc[i]
            else:
                row = data_scenarios.loc[i]

            fig.add_trace(go.Scatter(
                x = maturities_yf,
                y = row,
                name = label_curves_bulk,
                legendgroup = label_curves_bulk,
                mode = default_plotly_scatter_mode,
                showlegend = showlegend,
                line=dict(color="Black"),
                opacity = opacity#0.08
            ))
            showlegend = False

        showlegend = True
        for i in highlighted_indeces:
            if integer_index:
                row = data_scenarios.iloc[i]
            else:
                row = data_scenarios.loc[i]
                
            fig.add_trace(go.Scatter(
                x = maturities_yf,
                y = row,
                name = 'extreme scenarios',
                legendgroup = 'extreme scenarios',
                mode = default_plotly_scatter_mode,
                showlegend = showlegend
                ,line=dict(color=color_graphblue)
            ))
            showlegend = False

        fig.add_trace(go.Scatter(x = maturities_yf, y = data_current, name = 'current curve', mode = default_plotly_scatter_mode, line=dict(color="LightCyan"),))


        fig.update_layout(
            showlegend=True,
            xaxis = dict(title_text = "Expiry (in years)"),
            yaxis = dict(title_text = "Interest rate (in percent)"),
            legend = dict(traceorder='reversed'),
            title=get_default_title_dict(title_figure)
        )

        fig.show()

In [ ]:
# define a function that computes the 'distance' of the interest rate scenarios to given interest rate data and
# sorts the scenarios by that distance.
def computeDistancesAndSortDataFrame(
    data_current,
    data_scenarios,
    considered_columns,
    ascending = False,
    output_with_distances = False,
    distance_column_name = 'Distance'
):
    # compute the distances
    diffs = data_scenarios[considered_columns] - data_current[considered_columns]
    distances = [ np.linalg.norm(row, ord = 2) for index, row in diffs.iterrows() ]
    
    # add them as a column to a copy of the given scenario data
    data_scenarios_with_dist = data_scenarios.copy()
    data_scenarios_with_dist[distance_column_name] = distances
    
    # sort by distance
    data_scenarios_with_dist.sort_values(by = distance_column_name, ascending = ascending, inplace=True)
#     data_scenarios_with_dist = data_scenarios_with_dist.reset_index(drop=True) # use running number as index

    if output_with_distances:
        return data_scenarios_with_dist
    else:
        return data_scenarios_with_dist.drop(distance_column_name, axis=1)

In [ ]:
# Define a function that plots a histogram showing the interest rates of a given maturity
def displayHistogramForGivenTenor(
    data_scenarios,
    maturity_string
):
    if maturity_string in data_scenarios.columns:
        marker=dict(
            color=color_histmarker,
            line = dict(color = color_histmarkerborder, width = 1)
        )
        
        fig = go.Figure()
        fig.add_trace(go.Histogram(x = data_scenarios[maturity_string], marker = marker))

        fig.update_layout(
            showlegend=False,
            title = get_default_title_dict("Histogram for maturity \"{0}\"".format(maturity_string)),
            xaxis = dict(title_text = "Interest rate [%]"),
            yaxis = dict(title_text = "Number of occurences")
        )

        fig.show()    

In [ ]:
# Use widgets to make the plot interactive
def makeWidgetedHistogram(data_scenarios):
    combobox_maturity = widgets.Combobox(
        placeholder='Choose Maturity',
        options=[col for col in data_scenarios.columns],
        description='Maturity:',
        ensure_option=True,
        disabled=False
    )

    widgets.interact(
        lambda maturity: displayHistogramForGivenTenor(data_scenarios, maturity),
        maturity = combobox_maturity
    );

### EONIA

In [ ]:
# Plot the scenarios and highlight 'most distant' ones
disregarded_columns = ['1M', '2M', '4M', '5M', '7M', '8M', '10M', '11M']
considered_columns = [c for c in columns_maturity_EONIA if c not in disregarded_columns]

data_scenarios_EONIA_with_dist = computeDistancesAndSortDataFrame(
    data_EONIA_current,
    data_scenarios_EONIA,
    considered_columns,
    ascending = False,
    output_with_distances = True,
    distance_column_name = 'dist'
)

data_scenarios_EONIA_without_dist = data_scenarios_EONIA_with_dist.drop('dist', axis=1)

# We'll use this later
maxdist_hist = max(data_scenarios_EONIA_with_dist['dist'])

plotScenarios(
    data_EONIA_current,
    data_scenarios_EONIA_without_dist,
    maturities_EONIA_yf,
    title_figure = "Historical scenarios (EONIA)",
    label_curves_bulk = "other scenarios",
    highlighted_indeces = [0,1,2,3],
    integer_index = True,
    use_matplotlib = False
)

del data_scenarios_EONIA_without_dist
del disregarded_columns
del considered_columns

We can take a look at histograms showing the scenario data of specific tenors.

In [ ]:
makeWidgetedHistogram(data_scenarios_EONIA)

### EUR3M

In [ ]:
# Plot the scenarios and highlight some extreme ones
data_scenarios_EUR3M_with_dist = computeDistancesAndSortDataFrame(
    data_EUR3M_current,
    data_scenarios_EUR3M,
    columns_maturity_EUR3M,
    ascending = False,
    output_with_distances = True,
    distance_column_name = 'dist'
)

data_scenarios_EUR3M_without_dist = data_scenarios_EUR3M_with_dist.drop('dist', axis=1)

plotScenarios(
    data_EUR3M_current,
    data_scenarios_EUR3M_without_dist,
    maturities_EUR3M_yf,
    title_figure = "Historical scenarios (EUR3M)",
    label_curves_bulk = "other scenarios",
    highlighted_indeces = [0,5,7,8],
    integer_index = True,
    use_matplotlib = False
)

del data_scenarios_EUR3M_with_dist
del data_scenarios_EUR3M_without_dist

In [ ]:
makeWidgetedHistogram(data_scenarios_EUR3M)

### EUR6M

In [ ]:
# Plot the scenarios and highlight some extreme ones
# data_scenarios_EUR6M_with_dist = computeDistancesAndSortDataFrame(
#     data_EUR6M_current,
#     data_scenarios_EUR6M,
#     columns_maturity_EUR6M,
#     ascending = False,
#     output_with_distances = True,
#     distance_column_name = 'dist'
# )

# data_scenarios_EUR6M_without_dist = data_scenarios_EUR6M_with_dist.drop('dist', axis=1)

plotScenarios(
    data_EUR6M_current,
    data_scenarios_EUR6M,
    maturities_EUR6M_yf,
    title_figure = "Historical scenarios (EUR6M)",
    label_curves_bulk = "other scenarios"
#     ,highlighted_indeces = [0,1,2,3],
#     integer_index = True,
#     use_matplotlib = False
)

# del data_scenarios_EUR6M_with_dist
# del data_scenarios_EUR6M_without_dist

In [ ]:
makeWidgetedHistogram(data_scenarios_EUR6M)

In the case that the scenarios were constructed using relative historical changes, you'll probably find that some of them are rather extreme. To get a better understanding of why they are, we take a closer look at the scenarios containing the highest and the lowest interest rates found in any scenario.

In [ ]:
if scenario_construction_type == 'relative':
    # print(data_scenarios_EONIA.min())
    # print(data_scenarios_EONIA.idxmin())
    # print(data_scenarios_EONIA.min().min())
    # print(data_scenarios_EONIA.min().idxmin())
    # print(imin)

    imin = data_scenarios_EONIA.idxmin()[data_scenarios_EONIA.min().idxmin()]
    imax = data_scenarios_EONIA.idxmax()[data_scenarios_EONIA.max().idxmax()]

    display(
        pd.DataFrame({
            'Current': data_EONIA_current,
            'imin': data_EONIA_rates_only.loc[imin,:],
            'imin - n': data_EONIA_rates_only.loc[imin - timehorizon,:],
            'Scenario (imin)': data_scenarios_EONIA.loc[imin,:],
            'imax': data_EONIA_rates_only.loc[imax,:],
            'imax - n': data_EONIA_rates_only.loc[imax - timehorizon,:],
            'Scenario (imax)': data_scenarios_EONIA.loc[imax,:]
        }).head(len(data_EONIA_current))
    )

In [ ]:
# del data_EONIA
# del data_EONIA_bootstrapped
# del data_EONIA_current
# del data_EONIA_raw
# del data_scenarios_EONIA
# del data_scenarios_EONIA_diff
# del data_scenarios_EONIA_with_dist
# del maturities_EONIA_dates
# del maturities_EONIA_yf

# Monte Carlo simulation


In [ ]:
# Determine which curve data we are going to use in our MC simulation
data_scenarios_MC = data_scenarios_EUR6M
data_scenarios_MC_diff = data_scenarios_EUR6M_diff
data_MC_current = data_EUR6M_current
maturities_MC_yf = maturities_EUR6M_yf
maturities_MC_dates = maturities_EUR6M_dates
columns_maturity_MC = columns_maturity_EUR6M

## Random parallel shift
We apply a random parallel shift to the current interest rate curve.

In [ ]:
# Generate scenarios
n_sims_simpleMC = default_sample_size_MC
np.random.seed(7001)
random_shifts = np.random.normal(0, 0.02, n_sims_simpleMC)
data_scenarios_random_shift = pd.DataFrame().reindex_like(data_scenarios_MC)
data_scenarios_random_shift = data_scenarios_random_shift.iloc[0:0]

for x in random_shifts:
    data_scenarios_random_shift = data_scenarios_random_shift.append(data_MC_current + x, ignore_index=True)

# data_scenarios_random_shift.describe()

In [ ]:
# Plot scenarios
plotScenarios(
    data_MC_current,
    data_scenarios_random_shift,
    maturities_MC_yf,
    title_figure = 'Monte Carlo scenarios',
    label_curves_bulk = 'Monte Carlo scenarios',
    use_matplotlib = True
)

In [ ]:
makeWidgetedHistogram(data_scenarios_random_shift)

## Randomly picking historical data


### Using random indeces

In [ ]:
# Generate scenarios
n_picks_indeces = default_sample_size_MC
np.random.seed(7003)
num_hist_scenarios = len(data_scenarios_MC.index)
random_indeces= []

while len(random_indeces) < n_picks_indeces:
    sample = math.floor(abs(np.random.normal(0, num_hist_scenarios/1)))
    if sample >= 0 and sample < num_hist_scenarios:
        random_indeces.append(sample)

data_scenarios_random_pick_indeces = pd.DataFrame().reindex_like(data_scenarios_MC)
data_scenarios_random_pick_indeces = data_scenarios_random_pick_indeces.iloc[0:0]

for i in random_indeces:
    data_scenarios_random_pick_indeces = data_scenarios_random_pick_indeces.append(
        data_scenarios_MC.iloc[num_hist_scenarios-i-1],
        ignore_index=True
    )

In [ ]:
# Plot scenarios
plotScenarios(
    data_MC_current,
    data_scenarios_random_pick_indeces,
    maturities_MC_yf,
    title_figure = 'Monte Carlo scenarios',
    label_curves_bulk = 'Monte Carlo scenarios',
    use_matplotlib = True
)

In [ ]:
makeWidgetedHistogram(data_scenarios_random_pick_indeces)

## Using multivariate normal distribution

In [ ]:
# Determine the covariance matrix and mean vector
cov = data_scenarios_MC_diff.cov()
mean = data_scenarios_MC_diff.mean(axis='index')

In [ ]:
# Generate scenarios
np.random.seed(7007)
data_scenarios_random_multivariate_normal_diff = np.random.multivariate_normal(
    mean,
    cov,
    size = default_sample_size_MC,
    check_valid = 'raise'
)
data_scenarios_random_multivariate_normal_diff = pd.DataFrame(
    data = data_scenarios_random_multivariate_normal_diff,
    columns = data_scenarios_MC_diff.columns
)

data_scenarios_random_multivariate_normal = applyScenarios(
    data_MC_current,
    data_scenarios_random_multivariate_normal_diff,
    absolute = (scenario_construction_type == 'absolute')
)

# # Check quality
# display(((cov - data_scenarios_random_multivariate_normal_diff.cov())/cov).abs().describe())

In [ ]:
# Plot scenarios
plotScenarios(
    data_MC_current,
    data_scenarios_random_multivariate_normal,
    maturities_MC_yf,
    title_figure = 'Monte Carlo scenarios',
    label_curves_bulk = 'Monte Carlo scenarios',
    use_matplotlib = True
)

In [ ]:
makeWidgetedHistogram(data_scenarios_random_multivariate_normal)

## Simulation via short rate models

In [ ]:
# Create discount curve defined by the current interest rate data
discountcurve_HW = getDiscountCurve(
    'discountcurve_HW',
    refdate,
    maturities_MC_dates,
    data_MC_current,
    UnitInterestRate.PERCENT
)

In [ ]:
# hw_model = analytics.HullWhiteModel('HW_Model', refdate, 0.4, 0.002, discountcurve_HW) # EONIA
hw_model = analytics.HullWhiteModel('HW_Model', refdate, 0.6, 0.0008, discountcurve_HW)
hw_dc = model_tools.compute_yieldcurve(hw_model, refdate, maturities_MC_dates)

mkt_plot.curve(discountcurve_HW, maturities_MC_dates, refdate, True)
mkt_plot.curve(hw_dc, maturities_MC_dates, refdate, True)

In [ ]:
# # Plot interest rate curve
# fig = go.Figure()
# year_fractions = []
# for i in range(len(maturities_MC_dates)):
#     year_fractions.append(default_daycounter.yf(refdate, maturities_MC_dates[i]))
    
# values = [x for x in data_MC_current]

# fig.add_trace(go.Scatter(x = year_fractions, y = values, mode = 'lines+markers'))

In [ ]:
mkt_plot.curve(discountcurve_HW, maturities_MC_dates, refdate, False)
mkt_plot.curve(hw_dc, maturities_MC_dates, refdate, False)

In [ ]:
# # Plot discount curve based on current interest rate data and the yield curve produced by the hw_model
# fig = go.Figure()

# values = analytics.vectorDouble()
# discountcurve_HW.value(values, refdate, maturities_MC_dates)
# # convert to normal list
# values = [x for x in values]

# fig.add_trace(go.Scatter(x = year_fractions, y = values, mode = 'lines+markers'))
     
    
# values = analytics.vectorDouble()
# hw_dc.value(values, refdate, maturities_MC_dates)
# # convert to normal list
# values = [x for x in values]

# fig.add_trace(go.Scatter(x = year_fractions, y = values, mode = 'lines+markers'))

In [ ]:
# Generate scenarios using hw_model
sim_dates = converter.createPTimeList(refdate, maturities_MC_dates)
refdate_LTime = converter.getLTime(refdate)
n_sims = default_sample_size_MC
n_steps_per_year = 250
max_num_threads = 2

hw_lab = analytics.ModelLab(hw_model, refdate_LTime)
hw_lab.simulate(sim_dates, n_sims, n_steps_per_year, max_num_threads)

# sampling_points_EONIA_datediffdays = [math.ceil(365*yf) for yf in maturities_EONIA_yf]
# sim_dates = converter.createPTimeList(refdate, sampling_points_EONIA_datediffdays)


# for i in range(n_sims):
#     cir_lab.setFromSimulatedValues(cir, 1, i)  
#     dc = model_tools.compute_yieldcurve(cir, sim_dates[0], sampling_points_EONIA_datediffdays)    
#     mkt_plot.curve(dc, sim_dates, sim_dates[0], True, '', False)


# data_array_df = []
data_array_ir = []

for i in range(n_sims):
    hw_lab.setFromSimulatedValues(hw_model, 1, i)  
    dc = model_tools.compute_yieldcurve(hw_model, sim_dates[0], sim_dates)  
    daycounter = analytics.DayCounter(dc.getDayCounterType())
#     data_array_df.append(
#         [ dc.value(sim_dates[j], refdate) for j in range(len(sim_dates))]
#     )
    data_array_ir.append(
        getInterestRatesFromDiscountCurve(
            dc,
            refdate,
            sim_dates,
            UnitInterestRate.PERCENT
        )
    )
    
    # mkt_plot.curve(dc, sim_dates, sim_dates[0], True, '', False)

    
data_scenarios_hull_white = pd.DataFrame(
    data = data_array_ir,
    columns = columns_maturity_MC
)

# del data_array_df
del data_array_ir

In [ ]:
# Plot scenarios
plotScenarios(
    data_MC_current,
    data_scenarios_hull_white,
    maturities_MC_yf,
    title_figure = 'Monte Carlo scenarios',
    label_curves_bulk = 'Monte Carlo scenarios',
    use_matplotlib = True
)

In [ ]:
makeWidgetedHistogram(data_scenarios_hull_white)

# VaR computation

## Simple portfolio
To keep things simple, we start off with a portfolio containing only one fixed coupon bond with the following specifications:
 - It was issued on 2019/12/30
 - It has a maturity of 10 years
 - Its principal is 100€
 - It pays a 5€ coupon every year
 

In [ ]:
# Define the bond
maturity = 10
principal = 100.0
coupon_rate = 0.05
maturity_date = dt.datetime(year = refdate.year + maturity, month = refdate.month, day = refdate.day)
#print(refdate)
#print(maturity_date)

# Generate the coupon payment schedule as a vector of datetimes
coupon_dates = []
for i in range(maturity):
#     coupon_dates.append(dt.datetime(year = refdate.year + i, month = refdate.month, day = refdate.day) + dt.timedelta(days = 90))
#     coupon_dates.append(dt.datetime(year = refdate.year + i, month = refdate.month, day = refdate.day) + dt.timedelta(days = 180))
#     coupon_dates.append(dt.datetime(year = refdate.year + i, month = refdate.month, day = refdate.day) + dt.timedelta(days = 270))
    coupon_dates.append(dt.datetime(year = refdate.year + i + 1, month = refdate.month, day = refdate.day))
# print(coupon_dates)
coupon_rates = [coupon_rate]*len(coupon_dates)
# coupon_payments = [coupon_rate*principal]*len(coupon_dates)
# print(coupon_payments)

# We now use these specifications to define a fixed coupon bond
fixed_coupon_bond = pyvacon.instruments.BondSpecification('Fixed_Coupon', 'DBK', enums.SecuritizationLevel.NONE, 'EUR',
    maturity_date, refdate, principal, default_daycounter_type, coupon_dates, coupon_rates, '',[], [])

The current value of this portfolio can be computed by simply summing over all discounted future cash flows. Therefore, the only market variables affecting this value are the interest rates we use to determine the discount factors.

### Compute the Credit Spread and Portfolio Values

In [ ]:
# Define the pricer, we're going to use to price our bond
pricing_data_bond = pyvacon.pricing.BondPricingData()
pricing_data_bond.param = pyvacon.pricing.BondPricingParameter()
pricing_data_bond.param.useJLT = False
pricing_data_bond.pricingRequest = pyvacon.pricing.PricingRequest()
pricing_data_bond.pricingRequest.setCleanPrice(True)
pricing_data_bond.pricer = 'BondPricer'
pricing_data_bond.spec = fixed_coupon_bond

valdate = refdate # + dt.timedelta(days = timehorizon)
pricing_data_bond.valDate = valdate

In [ ]:
# Define functions that compute the values of the simple portfolio
def ComputeValueOfSimplePortfolio(
    refdate,
    maturities_dates,
    scenario,
    creditspread,
    unit_interest_rates,
    daycounter_type = default_daycounter_type,
    interpolation_type = default_interpolation_type,
    extrapolation_type = default_extrapolation_type
):
    # Add the credit spread we computed for our bond
    scenario = scenario + creditspread

    # Compute the price of the fixed coupon bond at the given valuation date
    pricing_data_bond.discountCurve = getDiscountCurve(
        'Discount Curve',
        refdate,
        maturities_dates,
        scenario,
        UnitInterestRate.PERCENT,
        daycounter_type,
        interpolation_type,
        extrapolation_type
    )
    results = pyvacon.pricing.price(pricing_data_bond)
    
    return [results.getPrice(), results.getCleanPrice()]


def ComputeValuesOfSimplePortfolio(
    refdate,
    maturities_dates,
    data_scenarios,
    creditspread,
    unit_interest_rates,
    daycounter_type = default_daycounter_type,
    interpolation_type = default_interpolation_type,
    extrapolation_type = default_extrapolation_type
):
    if not isinstance(data_scenarios, pd.DataFrame):
        raise Exception("Scenario data needs to be provided via a pandas.DataFrame.")
        
    # Compute the price of the fixed coupon bond at the given valuation date
    # Repeat for every scenario
    results_dirty = []
    results_clean = []
    
    for unused, scenario in data_scenarios.iterrows():
        results = ComputeValueOfSimplePortfolio(
            refdate,
            maturities_dates,
            scenario,
            creditspread,
            unit_interest_rates,
            daycounter_type,
            interpolation_type,
            extrapolation_type
        )
        
        results_dirty.append(results[0])
        results_clean.append(results[1])
        
    return [results_dirty, results_clean]

In [ ]:
# Choose the scenarios that we're going to use
data_scenarios = data_scenarios_EUR6M
maturities_dates = maturities_EUR6M_dates
data_current = data_EUR6M_current

Note: We are currently not taking portfolio aging into account. In the computations below, we are using the reference date as valuation date. That is, we look at the effects our shift scenarios would have on the value of our portfolio, if they were to happen instantaneously (instead of over the next $n$ days).

#### Compute Credit Spread
**Credit spread** is the difference in yield between two investments of similar maturities, but different credit qualities. It can be interpreted as the risk premium for one investment over the other.

Given the low EONIA rates, the bond we defined above currently has a much higher yield than a hypothetical bond paying EONIA rates on the same principal. Pricing the bond using discount factors based on EONIA rates would grossly overestimate its value. That is, the price we compute would be a lot higher than the bond's actual market value. To avoid this, we determine the constant shift we need to apply to the interest rates used for discounting in order for our price to equal the market value of the bond.

In [ ]:
# Use the current interest rates + a constant rate to compute the price of the fixed coupon bond
# Vary the constant rate and repeat until the value of the bond is right about the same as its principal
creditspread = coupon_rate * 100 # in percent
stepsize = coupon_rate * 100 # the initial step size used to vary the interest rate
spreads = []
values = []
for k in range(20):
    results = ComputeValueOfSimplePortfolio(
        refdate,
        maturities_dates,
        data_current,
        creditspread,
        UnitInterestRate.PERCENT
    )
    
    values.append(results[0])
    spreads.append(creditspread)
    
    if values[k] > principal:
        creditspread += stepsize
    else:
        creditspread -= stepsize
    stepsize /= 2

The credit spread is {{round(creditspread, 3)}}%.

In [ ]:
# Compute the current value without taking the credit spread into account
results = ComputeValueOfSimplePortfolio(
    refdate,
    maturities_dates,
    data_current,
    0,
    UnitInterestRate.PERCENT
)

current_value_without_credit_spread = results[0]

del results

In [ ]:
# Compute the current value
results = ComputeValueOfSimplePortfolio(
    refdate,
    maturities_dates,
    data_current,
    creditspread,
    UnitInterestRate.PERCENT
)

current_value = results[0]

del results

By taking into account the credit spread we computed above, we arrive at a current value of {{round(current_value, 6)}}, which closely matches the actual market value of 100. If we didn't take the credit spread into account, we would arrive at a value of {{round(current_value_without_credit_spread, 2)}}.

<a id='computePortfolioValues'></a>
#### Compute Portfolio Values

In [ ]:
# Compute the values of our simple portfolio in the generated scenarios
results, unused = ComputeValuesOfSimplePortfolio(
    refdate,
    maturities_dates,
    data_scenarios,
    creditspread,
    UnitInterestRate.PERCENT
)

### Plot pricing results

In [ ]:
# Define a function that plots the histrograms
def plotHistogram(
    data,
    binsstart,
    binsend,
    nbins,
    title_xaxis,
    markercolor = color_histmarker,
    bordercolor = color_histmarkerborder,
    title_figure = None
):
    xbins = dict(start = binsstart, end = binsend, size = (binsend-binsstart)/nbins)
    marker=dict(
        color=markercolor,
        line = dict(color = bordercolor, width = 1)
    )

    fig = go.Figure()
    fig.add_trace(go.Histogram(x = data, xbins = xbins, marker = marker))

    fig.update_layout(
        showlegend=False,
        xaxis = dict(title_text = title_xaxis, range = [binsstart, binsend]),
        yaxis = dict(title_text = "Number of occurences")
    )
    
    if title_figure != None and title_figure != "":
        fig.update_layout(
            title = get_default_title_dict(title_figure)
        )

    fig.show()  

In [ ]:
# # Histogramm of the pricing results
# plotHistogram(
#     data = results_dirty,
#     binsstart = xmin,
#     binsend = xmax,
#     nbins = 60,
#     title_xaxis = "Portfolio value"
# )

In [ ]:
# Histogramm of the changes/differences in value
diffs_value = np.asarray([res - current_value for res in results])
binsstart_simple = min(diffs_value)
binsend_simple = max(diffs_value)

plotHistogram(
    data = diffs_value,
    binsstart = binsstart_simple,
    binsend = binsend_simple,
    nbins = 60,
    title_xaxis = "Change in value",
    title_figure = "Changes in portfolio value"
)

<a id='computeVaR'></a>
### Compute value at risk
Let's recapitulate the definition of value at risk.

> Given a time horizon of $n$ days and a confidence level $\alpha$, the VaR is the loss of value that has the probability $\alpha$ not to be exceeded within the next $n$ days. In other words, the VaR is the $\alpha$-quantile of the distribution of loss in the value of a portfolio over the next $n$ days.

We already took the time horizon into account when we generated our scenarios. Let's assume we want $\alpha$ to be 99%. How do we determine the value at risk?

**Simple example:** If we have exactly 100 scenarios, then the 99%-quantile of the resulting cumulative relative frequency distribution is simply the second biggest loss occurring in our simulation. In other words, if we sort the losses in ascending order, the value at risk is the 99th one.

<a id='VaR_formula'></a>
**General case:**
Let $0 < \alpha < 1$ be the chosen confidence level, $n_s$ be the number of scenarios we generated and let the losses $l_1, \dots, l_{n_s}$ be sorted in ascending order, i.e. $l_i < l_j$ for all $i < j$. Then we determine the value at risk as follows.
$$\text{VaR} = \left( 1 - \lambda \right) \cdot l_{\lfloor n_s \cdot \alpha \rfloor} + \lambda \cdot l_{\lfloor n_s \cdot \alpha \rfloor + 1}$$
where $\lambda = \left( n_s \cdot \alpha - \lfloor n_s \cdot \alpha \rfloor \right)$.

*Note: In the case that $n_s \cdot \alpha$ is an integer, this formula reads $\text{VaR} = l_{n_s \cdot \alpha}$. It is, therefore, consistent with the simple example above.*

We already computed the changes in portfolio value in all scenarios we generated (see [above](#computePortfolioValues)). We will now use that information to determine and sort the losses in value. Afterwards, we can compute the value at risk using the formula above.

In [ ]:
# Define a function which displays info about scenarios with highest losses together with their cumulative relative frequencies
def getHTMLTableLossesFrequencies(
    losses_input,
    confidence_level,
    basic_table_only,
    compute_VaR,
    compute_ES_weights,
    highlight_ES_relevant
):
    if(confidence_level < 0 or confidence_level > 1):
        raise Exception("The confidence level needs to be a value between 0 and 1.")
        
    losses = np.sort(losses_input)
    _num_scenarios = len(losses)
    
    _index_ceil = np.ceil(_num_scenarios*confidence_level).astype(int) - 1 # first index where the cumul. rel. frequ. is above the threshold set by the chosen confidence level
    num_additional_rows = 2
    index_first_displayed = _index_ceil - num_additional_rows
    
    rangestart = index_first_displayed
    
    # Collect info about the scenarios with highest losses together with their cumulative relative frequency
    _data_scenario_names = [i for i in range(rangestart, _num_scenarios+1)]
    _data_crf = [100*i/_num_scenarios for i in range(rangestart, _num_scenarios+1)]
    _data_losses = ['{:.4f}'.format(losses[i]) for i in range(rangestart-1, _num_scenarios)]
    if not basic_table_only:
        _data_weights = ['-' if x <= 100*confidence_level else '1' for x in _data_crf]
    
    if not basic_table_only:
        _data_scenario_names += ['VaR']
        _data_crf += [confidence_level*100]
        if compute_VaR:
            _data_losses += ['{:.4f}'.format(computeVaR(confidence_level, losses_input))]
        else:
            _data_losses += ['?']
            
    _data = [
        _data_scenario_names,
        _data_crf,
        _data_losses
    ]
    _index = [
        'Scenario',
        'Cumulative relative frequency [%]',
        'Loss'
    ]
            
        
    if compute_ES_weights and not basic_table_only:
        unused, _weight_VaR = computeES(confidence_level, losses_input, output_weight_VaR=True)
        _data_weights += ['{:.4f}'.format(_weight_VaR)]
        _data += [_data_weights]
        _index += ['Weights in ES formula']
    
    
    # Put the info into a DataFrame
    df_table = pd.DataFrame(
        data = _data,
        index = _index
    ).transpose()
    
    # Sort by cumulative relative frequency and format floats afterwards
    df_table.sort_values(by = 'Cumulative relative frequency [%]', ascending = True, inplace = True)
    df_table['Cumulative relative frequency [%]'] = df_table['Cumulative relative frequency [%]'].map('{:7.3f}'.format)

    # Display and apply css to output
    tablestyles = [
        dict(selector='tr *', props=[('text-align', 'center')]),
        dict(selector='', props=[
            ('margin-left', 'auto'),
            ('margin-right', 'auto')
        ])
    ]
    
    if highlight_ES_relevant and not basic_table_only:
        # the parameter we're going to use with the nth-child CSS selector
        startat = num_additional_rows + 1 + 1
        
        # Add highlight to table styles
        tablestyles_highlight = [
            dict(
                selector='tbody tr:nth-child(n+{0})'.format(startat),
                props=[
                    ('border-left', '2px solid %s' % color_graphblue),
                    ('border-right', '2px solid %s' % color_graphblue)
                ]
            ),
            dict(
                selector='tbody tr:nth-child({0})'.format(startat),
                props=[('border-top', '2px solid %s' % color_graphblue)]
            ),
            dict(
                selector='tbody tr:nth-last-child(1)',
                props=[('border-bottom', '2px solid %s' % color_graphblue)]
            )
        ]
        
        tablestyles += tablestyles_highlight
    

    
    styler = df_table.style.set_table_styles(tablestyles).hide_index()
    
    return styler._repr_html_()
    
    
def displayTableLossesFrequencies(
    losses_input,
    confidence_level,
    basic_table_only = False,
    compute_VaR = False,
    compute_ES_weights = False,
    highlight_ES_relevant = False
):    
    html = getHTMLTableLossesFrequencies(
        losses_input = losses_input,
        confidence_level = confidence_level,
        basic_table_only = basic_table_only,
        compute_VaR = compute_VaR,
        compute_ES_weights = compute_ES_weights,
        highlight_ES_relevant = highlight_ES_relevant
    )
    display_html(html, raw = True)
    
    
# TODO
# def displayMultipleTablesLossesFrequencies(
#     losses_inputs,
#     indeces_first_displayed,
#     additional_tablestyles = None,
#     side_by_side = False
# ):    
#     if len(losses_inputs) != len(indeces_first_displayed):
#         raise Exception("The list losses_inputs must have the same length as indeces_first_displayed.")
    
#     if additional_tablestyles != None and len(additional_tablestyles) != len(losses_inputs):
#         raise Exception("The list additional_tablestyles, if supplied, must have the same length as losses_inputs and indeces_first_displayed.")
        
#     joint_html = ""
#     for i in range(len(losses_inputs)):
#         html = getHTMLTableLossesFrequencies(
#             losses_inputs[i],
#             indeces_first_displayed[i],
#             additional_tablestyles = additional_tablestyles[i] if additional_tablestyles != None else None
#         )
        
#         joint_html += html
        
#         if not side_by_side:
#             display_html(html, raw = True)
    
#     if side_by_side:
#         display_html(joint_html, raw = True)

In [ ]:
# Define a function which computes the value at risk
def computeVaR(
    confidence_level,
    losses_input
):
    if(confidence_level < 0 or confidence_level > 1):
        raise Exception("The confidence level needs to be a value between 0 and 1.")
        
    loss_scenarios = np.sort(losses_input)
    num_scenarios = len(loss_scenarios)
    
    # Compute the number of the entry corresponding to the confidence_level; round down if it's not an integer
    floor = np.floor(num_scenarios*confidence_level).astype(int)  
    
    # To get the index of this entry, we have to subtract 1
    floor_index = floor - 1
    
    # Compute VaR
    if(floor == num_scenarios):
        VaR = loss_scenarios[floor_index]
    elif(floor == 0):
        VaR = loss_scenarios[0]
    else:
        lam = num_scenarios * confidence_level - floor
        VaR = (1-lam) * loss_scenarios[floor_index] + lam * loss_scenarios[floor_index + 1]
    
    return VaR

In [ ]:
# Determine losses
alpha = 0.99

# Determine losses and sort in ascending order
losses = (-1)*diffs_value
losses = np.sort(losses)

# We will use the following variables in a markdown cell below
percentage = 100*alpha
num_scenarios = len(losses)
index_ceil = np.ceil(num_scenarios*alpha).astype(int) - 1 # first index where the cumul. rel. frequ. is above the threshold set by the chosen confidence level
probability_ceil = (index_ceil + 1)/num_scenarios*100
probability_floor = (index_ceil)/num_scenarios*100

Let us first take a look at the ordered list of losses and their respective cumulative relative frequencies.

In [ ]:
# Display scenarios with highest losses together with their cumulative relative frequency
displayTableLossesFrequencies(
    losses,
    confidence_level = alpha
)

Since we are working with an empirical probability distribution that is based on {{num_scenarios}} scenarios, we aren't able to match the given confidence level exactly. Looking at the cumulative relative frequencies of the highest losses we simulated (see table above), we can see that scenario {{index_ceil+1}} with a loss of {{round(losses\[index_ceil\], 4)}} is the first scenario to cross the threshold of {{percentage}}%.

Depending on the chosen confidence level and the number of scenarios, the cumulative relative frequencies immediately below and above the confidence level (in our case {{round(probability_floor, 4)}}% and {{round(probability_ceil, 4)}}%) can be close to the confidence level or quite far away. In order not to systematically underestimate or overestimate the value at risk, we linearly interpolate between these two scenarios (see <a href = '#VaR_formula'>formula above</a>).

In [ ]:
# Compute VaR
VaR_simple = computeVaR(alpha, losses)

The value at risk is {{round(VaR_simple, 4)}}.

In [ ]:
# Plot cumulative relative frequencies of loss of portfolio value
marker=dict(
    color=color_histmarker
)

fig = go.Figure()
fig.add_trace(go.Histogram(x = losses, nbinsx = len(losses)*2, marker = marker, cumulative_enabled = True, histnorm = 'probability', name = 'cumulative relative frequency distribution'))
fig.add_trace(go.Scatter(x = [VaR_simple, VaR_simple], y = [0, alpha], mode = 'lines', line = dict(color='Orange', width = 1), hoverinfo='skip'))
fig.add_trace(go.Scatter(x = [losses[0], VaR_simple], y = [alpha, alpha], mode = 'lines', line = dict(color='Orange', width = 1), hoverinfo='skip'))

fig.update_layout(
    showlegend=False,
    xaxis = dict(title_text = "Loss of portfolio value"),
    yaxis = dict(title_text = "Cumulative relative frequency")
)

fig.show()  

In [ ]:
# Clean up
del percentage
del probability_ceil
del probability_floor
del diffs_value

### Compute expected shortfall
The expected shortfall is the loss we expect to take given that the loss is bigger than or equal to the value at risk.

<b>Simple example:</b>
Let's again assume that we are working with 100 scenarios and a confidence level of 99%. As explained <a href = '#computeVaR'>above</a> the value at risk is then equal to the second biggest loss occurring in our scenarios. The expected shortfall is the average over the two biggest losses.

<b>General case:</b>
Let $0 < \alpha < 1$ be the chosen confidence level, $n_s$ be the number of scenarios we generated and let the losses $l_1, \dots, l_{n_s}$ be sorted in ascending order, i.e. $l_i < l_j$ for all $i < j$. Then we define the expected shortfall to be the following weighted average.
\begin{align}
\text{ES} &= \frac{1}{\left( \lfloor n_s \cdot \alpha \rfloor + 1 - n_s \cdot \alpha \right) + \left(n_s - \lfloor n_s \cdot \alpha \rfloor \right)}
\cdot \left(
\left( \lfloor n_s \cdot \alpha \rfloor + 1 - n_s \cdot \alpha \right) \cdot \text{VaR} \enspace
+
\sum_{i = \lfloor n_s \cdot \alpha \rfloor+1}^{n_s} l_i
\right) \\
&= \frac{1}{n_s - n_s \cdot \alpha + 1}
\cdot \left(
\left( \lfloor n_s \cdot \alpha \rfloor + 1 - n_s \cdot \alpha \right) \cdot \text{VaR} \enspace
+
\sum_{i = \lfloor n_s \cdot \alpha \rfloor+1}^{n_s} l_i
\right)
\end{align}

If $n_s \cdot \alpha$ is an integer, we have $\lfloor n_s \cdot \alpha \rfloor = n_s \cdot \alpha$ and $\text{VaR} = l_{n_s \cdot \alpha}$. In that case, the expected shortfall is simply the average over all losses bigger than or equal to $l_{n_s \cdot \alpha}$.
\begin{align}
\text{ES} 
&= \frac{1}{n_s - n_s \cdot \alpha + 1}
\cdot \left(
\left( \lfloor n_s \cdot \alpha \rfloor + 1 - n_s \cdot \alpha \right) \cdot \text{VaR} \enspace
+
\sum_{i = \lfloor n_s \cdot \alpha \rfloor+1}^{n_s} l_i
\right)\\
&= \frac{1}{n_s - n_s \cdot \alpha + 1}
\cdot \left(
\left( n_s \cdot \alpha + 1 - n_s \cdot \alpha \right) \cdot l_{n_s \cdot \alpha} \enspace
+
\sum_{i = n_s \cdot \alpha + 1}^{n_s} l_i
\right)\\
&= \frac{1}{n_s - n_s \cdot \alpha + 1}
\cdot \left(
1 \cdot l_{n_s \cdot \alpha} \enspace
+
\sum_{i = n_s \cdot \alpha + 1}^{n_s} l_i
\right)\\
&= \frac{1}{n_s - n_s \cdot \alpha + 1} \cdot
\sum_{i = n_s \cdot \alpha}^{n_s} l_i
\end{align}

In [ ]:
# Define a function which computes the expected shortfall
def computeES(
    confidence_level,
    losses_input,
    output_weight_VaR = False
):
    if(confidence_level < 0 or confidence_level > 1):
        raise Exception("The confidence level needs to be a value between 0 and 1.")
        
    loss_scenarios = np.sort(losses_input)
    num_scenarios = len(loss_scenarios)
    
    # Compute the number of the entry corresponding to the confidence_level; round down if it's not an integer
    floor = np.floor(num_scenarios*confidence_level).astype(int)  
    
    # To get the index of this entry, we have to subtract 1
    floor_index = floor - 1
    
    # Compute VaR
    if(floor == num_scenarios):
        ES = loss_scenarios[floor_index]
    elif(floor == 0):
        ES = np.average(loss_scenarios)
    else:
        w = floor + 1 - num_scenarios * confidence_level
        weights = [w] + ([1] * (num_scenarios - floor))
        summands = [computeVaR(confidence_level, losses_input)] + [loss_scenarios[i] for i in range(floor_index+1, len(loss_scenarios))]
        ES = np.average(summands, weights = weights)
    
    if output_weight_VaR:
        return ES, w
    else:
        return ES
    

In [ ]:
# Display the same table as above, but highlight the scenarios relevant for computing the expected shortfall
displayTableLossesFrequencies(
    losses,
    confidence_level = alpha,
    compute_VaR = True,
    compute_ES_weights = True,
    highlight_ES_relevant = True
)

In [ ]:
# Compute the expected shortfall
expected_shortfall_simple = computeES(alpha, losses)

The expected shortfall is {{round(expected_shortfall_simple, 4)}}.

In [ ]:
# Clean up
del alpha
del num_scenarios
del index_ceil

## Extended portfolio (add a swap)
We swap the fixed coupon payments for interest payments based on EONIA rates.

In [ ]:
# Define the swap scpecification
startdates = [refdate]
startdates.extend(coupon_dates[0:len(coupon_dates)-1])
#startdates = converter.createPTimeList(refdate, startdates)

enddates = coupon_dates
#enddates = converter.createPTimeList(enddates, startdates)

#print(startdates)
#print(enddates)

paydates = enddates
resetdates = startdates

notionals = analytics.vectorDouble()
notionals.append(principal)

fixedleg = analytics.IrFixedLegSpecification(coupon_rate, notionals, startdates, enddates, paydates,'EUR', default_daycounter_type)

floatleg = analytics.IrFloatLegSpecification(notionals, resetdates, startdates, enddates,
                                    paydates,'EUR', 'test_udl', default_daycounter_type, 
                                    0)
                                    #creditspread/100) # spread is given in percent

ir_swap = analytics.InterestRateSwapSpecification('TEST_SWAP', 'DBK', enums.SecuritizationLevel.COLLATERALIZED, 'EUR',
                                           converter.getLTime(paydates[-1]), fixedleg, floatleg)


### Recompute the value of our portfolio in all scenarios

In [ ]:
# Specify all data we need to price the swap
pricing_data_swap = analytics.InterestRateSwapPricingData()

pricing_data_pay_leg = analytics.InterestRateSwapLegPricingData()
pricing_data_pay_leg.spec = ir_swap.getPayLeg()
pricing_data_pay_leg.fxRate = 1.0
pricing_data_pay_leg.weight = -1.0

pricing_data_rec_leg = analytics.InterestRateSwapFloatLegPricingData()
pricing_data_rec_leg.spec = ir_swap.getReceiveLeg()
pricing_data_rec_leg.fxRate = 1.0
pricing_data_rec_leg.weight = 1.0

pricing_data_swap.pricer = 'InterestRateSwapPricer'
pricing_data_swap.pricingRequest = analytics.PricingRequest()
pricing_data_swap.valDate = converter.getLTime(refdate)
pricing_data_swap.setCurr('EUR')
pricing_data_swap.addLegData(pricing_data_pay_leg)
pricing_data_swap.addLegData(pricing_data_rec_leg)

In [ ]:
# Compute the price of our portfolio in every scenario
results_dirty = []
results_clean = []

# results_dirty_bond = []
# results_clean_bond = []

# results_dirty_swap = []
# results_clean_swap = []

for index, scenario in data_scenarios.iterrows():
    dc_scenario = getDiscountCurve(
        'dcScenario',
        refdate,
        maturities_dates,
        scenario,
        UnitInterestRate.PERCENT
    )
    
    dc_withspread = getDiscountCurve(
        'dcWithSpread',
        refdate,
        maturities_dates,
        scenario + creditspread,
        UnitInterestRate.PERCENT
    )
    
    pricing_data_bond.discountCurve = dc_scenario # dc_withspread
    pricing_data_pay_leg.discountCurve = dc_scenario # dc_withspread
    pricing_data_rec_leg.discountCurve = dc_scenario
    pricing_data_rec_leg.fixingCurve = dc_scenario
    
    price_bond = pyvacon.pricing.price(pricing_data_bond)
    price_swap = analytics.price(pricing_data_swap)
    results_dirty.append(price_bond.getPrice() + price_swap.getPrice())
    results_clean.append(price_bond.getCleanPrice() + price_swap.getCleanPrice())
    
#     results_dirty_bond.append(price_bond.getPrice())
#     results_dirty_swap.append(price_swap.getPrice())
    #print(pricing_data_bond.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()))
#print(results_dirty)
del dc_scenario
del dc_withspread
del price_bond
del price_swap

### Compute the current value as a reference

In [ ]:
# Define a discount curve based on the current rates
dc_current = getDiscountCurve(
    'dcCurrent',
    refdate,
    maturities_dates,
    data_current,
    UnitInterestRate.PERCENT
)

dc_withspread = getDiscountCurve(
    'dcWithSpread',
    refdate,
    maturities_dates,
    data_current + creditspread,
    UnitInterestRate.PERCENT
)

pricing_data_bond.discountCurve = dc_current # dc_withspread
pricing_data_pay_leg.discountCurve = dc_current # dc_withspread
pricing_data_rec_leg.discountCurve = dc_current 
pricing_data_rec_leg.fixingCurve = dc_current

# Compute portfolio value
price_bond = pyvacon.pricing.price(pricing_data_bond)
price_swap = analytics.price(pricing_data_swap)
#print(prSwap.getPrice())
#print(price_bond.getPrice())
value_bond_current = price_bond.getPrice()
value_swap_current = price_swap.getPrice()
value_portfolio_current = price_bond.getPrice() + price_swap.getPrice()
#print(value_swap_current)
#print(value_bond_current)
#print(value_portfolio_current)

del dc_current
del dc_withspread
del price_bond
del price_swap
# del value_bond_current # used later
# del value_swap_current # used later

### Plot the pricing results

In [ ]:
# Histogramm of the changes/differences in value
diffs_value_dirty = np.asarray([res - value_portfolio_current for res in results_dirty])
plotHistogram(
    data = diffs_value_dirty,
    binsstart = binsstart_simple,
    binsend = binsend_simple,
    nbins = 60,
    title_xaxis = 'Change in portfolio value'
)

# # Compare changes in Bond and Swap value
# diffs_value_dirty = np.asarray([res - value_bond_current for res in results_dirty_bond])
# plotHistogram(
#     data = diffs_value_dirty,
#     binsstart = min(diffs_value_dirty),
#     binsend = max(diffs_value_dirty),
#     nbins = 60,
#     title_xaxis = 'Change in portfolio value (bond only)'
# )

# diffs_value_dirty = np.asarray([res - value_swap_current for res in results_dirty_swap])
# plotHistogram(
#     data = diffs_value_dirty,
#     binsstart = min(diffs_value_dirty),
#     binsend = max(diffs_value_dirty),
#     nbins = 60,
#     title_xaxis = 'Change in portfolio value (swap only)'
# )

We can see that the swap we added to our portfolio cancels out any market risk, setting the value at risk to 0.

In [ ]:
# Clean up
del diffs_value_dirty
# del value_portfolio_current
del results_clean
del results_dirty

# del diffs_value_dirty
# del results_clean_bond
# del results_dirty_bond
# del results_clean_swap
# del results_dirty_swap

We can see that the swap we added to our portfolio cancels out any market risk, setting the value at risk to 0.

## Interest Rate Shock Scenarios

In [ ]:
# define the parameters for the shock scenarios

shockParams = pd.DataFrame({'Currency': [], 'Parallel': [], 'Short': [], 'Long': []})
shockParams = shockParams.append({'Currency': 'EUR', 'Parallel': 200, 'Short': 250, 'Long': 100}, ignore_index = True)
shockParams = shockParams.append({'Currency': 'GBP', 'Parallel': 250, 'Short': 300, 'Long': 150}, ignore_index = True)
shockParams = shockParams.append({'Currency': 'USD', 'Parallel': 200, 'Short': 300, 'Long': 150}, ignore_index = True)

### Compute the change in value

In [ ]:
# Compute the price of our portfolio
# Repeat for every scenario
results_dirty = []
results_clean = []
results_dirty_bondonly = []
results_clean_bondonly = []

currency = 'EUR'
parallel = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Parallel']
short = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Short']
long = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Long']
    
shockScenarios = ['ParallelUp', 'ParallelDown', 'ShortUp', 'ShortDown', 'LongUp', 'LongDown', 'Flatten', 'Steepen']

for shockScenario in shockScenarios:
    dc_current = getShockedDiscountCurve(
        'dc_current',
        refdate,
        maturities_dates,
        data_current*100,
        default_daycounter_type,
        default_interpolation_type,
        default_extrapolation_type,
        shockScenario,
        parallel,
        short,
        long
    )
    
#     dc_withspread = getShockedDiscountCurve(
#          'dc_withspread',
#          refdate,
#          maturities_dates,
#          data_current + creditspread,
#          default_daycounter_type,
#          default_interpolation_type,
#          default_extrapolation_type,
#          shockScenario,
#          parallel/100,
#          short/100,
#          long/100
#      )
    
    pricing_data_bond.discountCurve = dc_current # dc_withspread
    pricing_data_pay_leg.discountCurve = dc_current
    pricing_data_rec_leg.discountCurve = dc_current
    pricing_data_rec_leg.fixingCurve = dc_current
    
    price_bond = pyvacon.pricing.price(pricing_data_bond)
    price_swap = analytics.price(pricing_data_swap)
    dirty = price_bond.getPrice() + price_swap.getPrice()
    clean = price_bond.getCleanPrice() + price_swap.getCleanPrice()
    results_dirty.append(dirty)
    results_clean.append(clean)
    results_dirty_bondonly.append(price_bond.getPrice())
    results_clean_bondonly.append(price_bond.getCleanPrice())
    #print(pricing_data_bond.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()))
#print(results_dirty)

### Plot the change in value (comparison between our entire portfolio and the bond on its own)

In [ ]:
# Bar plot of the changes/differences in value
diffs_values_dirty = np.asarray([res - value_portfolio_current for res in results_dirty])
diffs_values_dirty_bondonly = np.asarray([res - value_bond_current for res in results_dirty_bondonly])

y_shockscenarios_simple = diffs_values_dirty_bondonly/value_bond_current*100
ymin = min(y_shockscenarios_simple)
ymax = max(y_shockscenarios_simple)
ydiff = abs(ymax-ymin)
rangemin = ymin - ydiff/10
rangemax = ymax + ydiff/10

marker=dict(
    color=color_histmarker,
    line = dict(color = color_histmarker, width = 1)
)



# Plot results for the simple portfolio

fig = go.Figure()
fig.add_trace(go.Bar(x=shockScenarios, y=y_shockscenarios_simple, marker = marker))

fig.update_layout(
    showlegend=False,
    xaxis = dict(title_text = "Shock Scenario"),
    yaxis = dict(title_text = "Change in portfolio value [%]", range = [rangemin, rangemax])
    ,title=get_default_title_dict("Simple Portfolio")
)

fig.show()  



# Plot results for the extended portfolio

fig = go.Figure()
fig.add_trace(go.Bar(x=shockScenarios, y=diffs_values_dirty/value_portfolio_current*100, marker = marker))

fig.update_layout(
    showlegend=False,
    xaxis = dict(title_text = "Shock Scenario"),
    yaxis = dict(title_text = "Change in portfolio value [%]", range = [rangemin, rangemax])
    ,title=get_default_title_dict("Extended Portfolio")
)

fig.show()  


del y_shockscenarios_simple, ymin, ymax, ydiff, rangemin, rangemax

In [ ]:
# Clean up
del results
del results_clean
del results_clean_bondonly
del results_dirty
del results_dirty_bondonly
del current_value
del current_value_without_credit_spread
del binsend_simple
del binsstart_simple
del losses
del data_current
del data_scenarios
del maturities_dates
del expected_shortfall_simple
del VaR_simple

In [ ]:
del diffs_values_dirty

In [ ]:
del diffs_values_dirty_bondonly

# Compare different methods

In [ ]:
# Choose the scenarios that we're going to use

# Main scenarios
data_scenarios = data_scenarios_MC
maturities_dates = maturities_MC_dates
data_current = data_MC_current

# Comparative scenarios
data_scenarios_compare = data_scenarios_random_multivariate_normal
# data_scenarios_compare = data_scenarios_random_pick_indeces
# data_scenarios_compare = data_scenarios_random_pick_dist
# data_scenarios_compare = data_scenarios_random_shift
# data_scenarios_compare = data_scenarios_random_buckets
# data_scenarios_compare = data_scenarios_buckets
# data_scenarios_compare = data_scenarios_hull_white

In [ ]:
# Set confidence level alpha
alpha = 0.99

## Changes in portfolio value

In [ ]:
# Compute the values of our simple portfolio in the chosen scenarios
results_main, unused = ComputeValuesOfSimplePortfolio(
    refdate,
    maturities_dates,
    data_scenarios,
    creditspread,
    UnitInterestRate.PERCENT
)

results_compare, unused = ComputeValuesOfSimplePortfolio(
    refdate,
    maturities_dates,
    data_scenarios_compare,
    creditspread,
    UnitInterestRate.PERCENT
)

current_value, unused = ComputeValueOfSimplePortfolio(
    refdate,
    maturities_dates,
    data_current,
    creditspread,
    UnitInterestRate.PERCENT
)

In [ ]:
# Since we want to compare scenarios, we determine the minimal and maximal x-values to display in upcoming plots
xmin = min(min(results_main), min(results_compare))
xmax = max(max(results_main), max(results_compare))
binsstart_simple = xmin - current_value
binsend_simple = xmax - current_value

In [ ]:
# Histogramm of the changes/differences in value
diffs_value_main = np.asarray([res - current_value for res in results_main])

plotHistogram(
    data = diffs_value_main,
    binsstart = binsstart_simple,
    binsend = binsend_simple,
    nbins = 60,
    title_xaxis = "Change in portfolio value",
    title_figure = "Main scenarios"
)


diffs_value_compare = np.asarray([res - current_value for res in results_compare])

plotHistogram(
    data = diffs_value_compare,
    binsstart = binsstart_simple,
    binsend = binsend_simple,
    nbins = 60,
    title_xaxis = "Change in portfolio value",
    title_figure = "Comparative scenarios"
)

In [ ]:
# Determine losses
losses_main = (-1)*diffs_value_main
losses_compare = (-1)*diffs_value_compare

In [ ]:
# Display main scenarios with highest losses together with their cumulative relative frequency
displayTableLossesFrequencies(
    losses_main,
    alpha,
    basic_table_only=True
)

In [ ]:
# Same for comparative scenarios
displayTableLossesFrequencies(
    losses_compare,
    alpha,
    basic_table_only=True
)

## Value at risk

In [ ]:
# Compute VaR
VaR_main = computeVaR(alpha, losses_main)
VaR_compare = computeVaR(alpha, losses_compare)

The value at risk of the main scenarios is {{round(VaR_main, 4)}}.
<br>
The value at risk of the comparative scenarios is {{round(VaR_compare, 4)}}.

In [ ]:
# Clean up
del diffs_value_main
del VaR_main

del diffs_value_compare
del VaR_compare

## Expected shortfall

In [ ]:
# Compute ES
expected_shortfall_main = computeES(alpha, losses_main)
expected_shortfall_compare = computeES(alpha, losses_compare)

The expected shortfall of the main scenarios is {{round(expected_shortfall_main, 4)}}.
<br>
The value at risk of the comparative scenarios is {{round(expected_shortfall_compare, 4)}}.

In [ ]:
# clean up
del alpha

del losses_main
del expected_shortfall_main

del losses_compare
del expected_shortfall_compare

# Distinguish different risk factors
In this section we want to consider the EUR6M curve as being composed of the EUR3M plus a basis spread curve.
We will compute the VaR of our simple portfolio for both of these factors and look at how they relate to each other and the VaR we computed above.

*Hinweis an mich selbst: Wir verwenden hier die Schnittmenge der Maturities, fuer die Daten fuer beide Kurven zur Verfuegung stehen. Bei den aktuellen Daten sind dies genau die Stuetzstellen der EUR6M, da diese komplett in denen fuer die EUR3M enthalten sind. In unserem Fall hat die 'Einschraekung' auf die Schnittmenge also keinen Effekt, koennte in einer anderen Situation aber durchaus einen Unterschied machen.*

In [ ]:
# EUR3M scenario + current basis spread
data_scenarios_ir = data_scenarios_EUR3M[columns_maturity_EUR3M_EUR6M].copy()
for i in range(len(data_scenarios_ir.index)):
    data_scenarios_ir.iloc[i] = data_scenarios_ir.iloc[i] + spreads_EUR3M_EUR6M_current

In [ ]:
# Current EUR3M + basis spread scenario
data_scenarios_bs = spreads_scenarios_EUR3M_EUR6M.copy()
for i in range(len(data_scenarios_bs)):
    data_scenarios_bs.iloc[i] = data_scenarios_bs.iloc[i] + data_EUR3M_current[columns_maturity_EUR3M_EUR6M]

In [ ]:
# EUR3M scenario + basis spread scenario (i.e. both factors together)
data_scenarios_total = data_scenarios_EUR3M[columns_maturity_EUR3M_EUR6M].copy() + spreads_scenarios_EUR3M_EUR6M.copy()
# data_scenarios_total = data_EUR6M_current[columns_maturity_EUR3M_EUR6M]
# display((data_scenarios_total - data_scenarios_EUR6M[columns_maturity_EUR3M_EUR6M]).abs().describe())

In [ ]:
# Compute the portfolio value in each if these scenarios
results_ir, unused = ComputeValuesOfSimplePortfolio(
    refdate,
    maturities_EUR3M_EUR6M_dates,
    data_scenarios_ir,
    creditspread,
    UnitInterestRate.PERCENT
)

results_bs, unused = ComputeValuesOfSimplePortfolio(
    refdate,
    maturities_EUR3M_EUR6M_dates,
    data_scenarios_bs,
    creditspread,
    UnitInterestRate.PERCENT
)

results_total, unused = ComputeValuesOfSimplePortfolio(
    refdate,
    maturities_EUR3M_EUR6M_dates,
    data_scenarios_total,
    creditspread,
    UnitInterestRate.PERCENT
)

In [ ]:
# Compute the current portfolio value
value_current, unused = ComputeValueOfSimplePortfolio(
    refdate,
    maturities_EUR3M_EUR6M_dates,
    data_EUR6M_current[columns_maturity_EUR3M_EUR6M],
    creditspread,
    UnitInterestRate.PERCENT
)

In [ ]:
# Histogramm of the changes/differences in value
diffs_value_ir = np.asarray([res - value_current for res in results_ir])
binsstart_simple_ir = min(results_ir) - value_current
binsend_simple_ir = max(results_ir) - value_current

plotHistogram(
    data = diffs_value_ir,
    binsstart = binsstart_simple_ir,
    binsend = binsend_simple_ir,
    nbins = 60,
    title_xaxis = "Change in portfolio value (IR)"
)


diffs_value_bs = np.asarray([res - value_current for res in results_bs])
binsstart_simple_bs = min(results_bs) - value_current
binsend_simple_bs = max(results_bs) - value_current

plotHistogram(
    data = diffs_value_bs,
    binsstart = binsstart_simple_bs,
    binsend = binsend_simple_bs,
    nbins = 60,
    title_xaxis = "Change in portfolio value (BS)"
)


diffs_value_total = np.asarray([res - value_current for res in results_total])
binsstart_simple_total = min(results_total) - value_current
binsend_simple_total = max(results_total) - value_current

# plotHistogram(
#     data = diffs_value_total,
#     binsstart = binsstart_simple_total,
#     binsend = binsend_simple_total,
#     nbins = 60,
#     title_xaxis = "Change in portfolio value (total)"
# )

In [ ]:
# Compute value at risk and expected shortfall
alpha = 0.99
percentage = 100*alpha

# Determine losses and sort in ascending order
diffs_value_ir = (-1)*np.sort((-1)*diffs_value_ir)
losses_ir = -diffs_value_ir
num_scenarios_ir = len(losses_ir)
index_ceil_ir = np.ceil(num_scenarios_ir*alpha).astype(int) - 1

VaR_ir = computeVaR(alpha, losses_ir)
expected_shortfall_ir = computeES(alpha, losses_ir)


# Determine losses and sort in ascending order
diffs_value_bs = (-1)*np.sort((-1)*diffs_value_bs)
losses_bs = -diffs_value_bs
num_scenarios_bs = len(losses_bs)
index_ceil_bs = np.ceil(num_scenarios_bs*alpha).astype(int) - 1

VaR_bs = computeVaR(alpha, losses_bs)
expected_shortfall_bs = computeES(alpha, losses_bs)


# Determine losses and sort in ascending order
diffs_value_total = (-1)*np.sort((-1)*diffs_value_total)
losses_total = -diffs_value_total
num_scenarios_total = len(losses_total)
index_ceil_total = np.ceil(num_scenarios_total*alpha).astype(int) - 1

VaR_total = computeVaR(alpha, losses_total)
expected_shortfall_total = computeES(alpha, losses_total)

The {{timehorizon}}-day {{percentage}}%-VaR regarding changes in the reference index is {{round(VaR_ir, 4)}}. The expected shortfall is {{round(expected_shortfall_ir, 4)}}.

The {{timehorizon}}-day {{percentage}}%-VaR regarding changes in basis spreads is {{round(VaR_bs, 4)}}. The expected shortfall is {{round(expected_shortfall_bs, 4)}}.

Taking both changes into account simultaneously, the {{timehorizon}}-day {{percentage}}%-VaR is {{round(VaR_total, 4)}} and the expected shortfall is {{round(expected_shortfall_total, 4)}}.

In [ ]:
# Display scenarios with highest losses together with their cumulative relative frequency (regarding changes in reference index)
displayTableLossesFrequencies(
    losses_ir,
    alpha,
    basic_table_only=True
)

In [ ]:
# Do the same for the changes in basis spread
displayTableLossesFrequencies(
    losses_bs,
    alpha,
    basic_table_only=True
)

In [ ]:
# And again, taking both factors into account
displayTableLossesFrequencies(
    losses_total,
    alpha,
    basic_table_only=True
)

In [ ]:
# Clean up
del VaR_ir, VaR_bs, VaR_total
del alpha, index_ceil_ir, index_ceil_bs, index_ceil_total, percentage
del binsstart_simple_ir, binsstart_simple_bs, binsend_simple_ir, binsend_simple_bs, binsend_simple_total, binsstart_simple_total
del results_ir, results_bs, results_total
del diffs_value_ir, diffs_value_bs, diffs_value_total
del losses_ir, losses_bs, losses_total

# TODO

- Reread the whole notebook and alter text according to recent changes
- Differenz OIS <-> 6M als Summe der beiden anderen Differenzen?
- Entwicklung von VaR und ES darstellen (sich bewegendes Zeitfenster)
- check use of iloc vs loc
- 'Einheit' der Zinssaetze mit an die Funktionen uebergeben (dezimal, percent, basispoint) // in Arbeit
- Das Wort 'current' als Beschreibung für die jüngste in den Marktdaten vorhandene EONIA-Kurve ueberdenken
- Actually order the historical data by date ascending (instead of descending) to avoid confusion
- Illustrate the definition of VaR
- Portfolio mit zwei Bonds unterschiedlicher Laufzeiten untersuchen
- highlight extreme scenarios for EUR6M
- Business day / holiday calendars berücksichtigen
- maturities als yf mittels dateutils.relativedelta und daycounter ausrechnen
- auch bei Bond Spec sinnvoll einsetzbar
- getBoostrappedData not isinstance(discount_curves, pd.DataFrame) darf auch pd.Series sein
- RefDates bei losses mitziehen
- ~Histrogramme für einzelne Stützstellen bei HistSim und MC~
- ~startat = 5 überprüfen~
- Hintergrundinfo zu den Shock Scenarios
- ~PlotMonteCarloScenarios und plotHistoricalScenarios zusammenführen~
- Bei unterschiedlichen Laufzeiten in den Quotes Interpolation möglich machen, um Spreads ausrechnen zu können
- Fixing bei Swap
- ~MC mit Varianz Kovarianz, multivariat, kovarianzmatrix für Änderungen zu allen Stützstellen, Cholesky-Zerlegung~
- ~Vergleich der Ergebnisse unterschiedlicher Verfahren der Szenarioerzeugung in separaten Abschnitt auslagern~
- Interest Rate Shock Scenarios wieder in eigenen Abschnitt auslagern (?)
- Hull-White kalibrieren
- Wie gehen wir mit Lücken in den Daten um?
- ~Berechnung von VaR und ES überall anpassen (hinterher mit Stefan vergleichen)~
- ~Tabelle mit Szenarioinformationen zur Erklaerung von ES erweitern?~
- Moeglichkeit schaffen Strings wie '1Y' und '6M' in eine Zeitspanne umzuwandeln

$\ell_n = ln(v_{n+1}) - \ln(v_n) = ln\left(\frac{v_{n+1}}{v_n}\right)$

$v_k = v \exp(\ell_k)$